<img align="right" src="images/ninologo.png" width="150"/>
<img align="right" src="images/tf-small.png" width="125"/>
<img align="right" src="images/dans.png" width="150"/>

# Search

Search is essential to get around in the corpus, and it is convenient as well.
Whereas the whole point of Text-Fabric is to move around in the corpus programmatically,
we show that
[template based search](https://dans-labs.github.io/text-fabric/Use/Search/#search-templates)
makes everything a lot more convenient ...

Along with showing how search works, we also point to pretty ways to display your search results.
The good news is that `search` and `pretty` work well together. 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
import collections
from IPython.display import display, Markdown
from tf.app import use

In [3]:
A = use('uruk', hoist=globals())

Using Nino-cunei/uruk/tf/uruk - 1.0 r1.2 in /Users/dirk/text-fabric-data


**Documentation:** <a target="_blank" href="https://github.com/Nino-cunei/uruk/blob/master/docs/" title="provenance of Uruk IV/III: Proto-cuneiform tablets ">URUK</a> <a target="_blank" href="https://github.com/Nino-cunei/uruk/blob/master/docs/transcription.md" title="How TF features represent ATF">Character table</a> <a target="_blank" href="https://github.com/Nino-cunei/uruk/blob/master/docs//transcription.md" title="URUK feature documentation">Feature docs</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Apps/Uruk/" title="uruk API documentation">uruk API</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/General/" title="text-fabric-api">Text-Fabric API 7.1.1</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Use/Search/" title="Search Templates Introduction and Reference">Search Reference</a>

Using Nino-cunei/uruk/sources/cdli/images r1.2 in /Users/dirk/text-fabric-data
Found 2095 ideograph linearts
Found 2724 tablet linearts
Found 5495 tablet photos


In [4]:
query = '''
tablet
  sign type=numeral
'''

results = A.search(query)

  0.26s 38122 results


In [5]:
A.table(results, condensed=True, condenseType='face', end=5)

n | p | face | sign
--- | --- | --- | ---
1| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006427" title="to CDLI main page for this tablet" sec="P006427">P006427</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006427" title="to CDLI main page for this tablet" sec="P006427">face obverse</a>|<span  class="hl" >3(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
2| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006428" title="to CDLI main page for this tablet" sec="P006428">P006428</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006428" title="to CDLI main page for this tablet" sec="P006428">face obverse</a>|<span  class="hl" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >3(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >1(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >1(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
3| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448701" title="to CDLI main page for this tablet" sec="P448701">P448701</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448701" title="to CDLI main page for this tablet" sec="P448701">face obverse</a>|<span  class="hl" >2(N19)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N19).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n19).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >4(N41)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/4(N41).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-4(n41).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >1(N46)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N46).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n46).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
4| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">P448702</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">face obverse</a>|<span  class="hl" >3(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >N</span> |<span  class="hl" >6(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/6(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-6(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >4(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/4(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-4(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >2(N39~a)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N39~a).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n39-a).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >N</span> |<span  class="hl" >N</span> |<span  class="hl" >2(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >3(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
5| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448703" title="to CDLI main page for this tablet" sec="P448703">P448703</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448703" title="to CDLI main page for this tablet" sec="P448703">face obverse</a>|<span  class="hl" >1(N08)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N08).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n08).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >1(N08)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N08).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n08).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >1(N08)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N08).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n08).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >1(N08)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N08).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n08).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >1(N08)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N08).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n08).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>

In [8]:
A.show(results, condenseType='face', end=5)



**tablet** *1*





**tablet** *2*





**tablet** *3*





**tablet** *4*





**tablet** *5*



## Finding a tablet

Suppose we have the *p-number* of a tablet.
How do we find that tablet?
Remembering from the feature docs that the p-numbers are stored in the feature
`catalogId`, we can write a *search template*.

In [9]:
query = '''
tablet catalogId=P005381
'''
results = A.search(query)
A.table(results)

  0.01s 1 result


n | p | tablet
--- | --- | ---
1| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P005381" title="to CDLI main page for this tablet" sec="P005381">P005381</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P005381" title="to CDLI main page for this tablet" sec="P005381">tablet P005381</a>

The function `A.table()` gives you a tabular overview of the results,
with a link to the tablet on CDLI.

But we can also get more information by using `A.show()`:

In [10]:
A.show(results)



**tablet** *1*



Several things to note here

* if you want to see the tablet on CDLI, you can click on the tablet header;
* the display matches the layout on the tablet:
  * faces and columns are delineated with red lines
  * lines and cases are delineated with blue lines
  * cases and subcases alternate their direction of division between horizontal and vertical:
    lines are horizontally divided into cases, they are vertically divided into subcases, and they
    in turn are horizontally divided in subsubcases, etc.
  * quads and signs are delineated with grey lines
  * clusters are delineated with brown lines (see further on)
  * lineart is given for top-level signs and quads; those that are part of a bigger quad do not
    get lineart;
    
It is possible to switch off the lineart.

## More info in the results
You can show the line numbers that correspond to the ATF source files as well.
Let us also switch off the lineart.

In [11]:
query = '''
tablet catalogId=P005381
'''
results = A.search(query)
A.table(results, lineNumbers=True)
A.show(results, lineNumbers=True, lineart=False)

  0.01s 1 result


n | p | tablet
--- | --- | ---
1| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P005381" title="to CDLI main page for this tablet" sec="P005381">P005381</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P005381" title="to CDLI main page for this tablet" sec="P005381">tablet P005381</a> @85111 



**tablet** *1*



There is a big quad in `obverse:2 line 1`. We want to call up the lineart for it separately.
First step: make the nodes visible.

In [12]:
query = '''
tablet catalogId=P005381
'''
results = A.search(query)
A.table(results, withNodes=True)
A.show(results, withNodes=True, lineart=False)

  0.01s 1 result


n | p | tablet
--- | --- | ---
1| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P005381" title="to CDLI main page for this tablet" sec="P005381">P005381</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P005381" title="to CDLI main page for this tablet" sec="P005381">tablet P005381</a> *148166* 



**tablet** *1*



We read off the node number of that quad and fetch the lineart.

In [13]:
A.lineart(143015)

## Search templates
Let's highlight all numerals on the tablet.

We prefer our results to be condensed per tablet for the next few shows.

We make that the temporary default:

In [23]:
A.displaySetup(condensed=True)

In [24]:
query = '''
tablet catalogId=P005381
  sign type=numeral
'''
results = A.search(query)
A.show(results)

  0.20s 10 results




**tablet** *1*



We can do the same for multiple tablets. But now we highlight the undivided lines,
just for variation.

In [25]:
query = f'''
tablet catalogId=P003581|P000311
    line terminal
'''
results = A.search(query)

  0.07s 11 results


In [26]:
A.table(results)

n | p | tablet | line | line | line | line | line | line
--- | --- | --- | --- | --- | --- | --- | --- | ---
1| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000311" title="to CDLI main page for this tablet" sec="P000311">P000311</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000311" title="to CDLI main page for this tablet" sec="P000311">tablet P000311</a>|<span  class="hldot" >line 1</span>|<span  class="hldot" >line 2</span>|<span  class="hldot" >line 1</span>|<span  class="hldot" >line 2</span>|<span  class="hldot" >line 3</span>|<span  class="hldot" >line 1</span>
2| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003581" title="to CDLI main page for this tablet" sec="P003581">P003581</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003581" title="to CDLI main page for this tablet" sec="P003581">tablet P003581</a>|<span  class="hldot" >line 1</span>|<span  class="hldot" >line 2</span>|<span  class="hldot" >line 2</span>|<span  class="hldot" >line 1</span>|<span  class="hldot" >line 2</span>

In [27]:
A.show(results, lineart=False)



**tablet** *1*





**tablet** *2*



In an other chapter of this tutorial, [steps](steps.ipynb) we encounter a grapheme with a double prime.
There is only one, and we showed the tablet on which it occurs, without highlighting the grapheme in question.
Now we can do the highlight:

In [28]:
results = A.search('''
sign prime=2
''')

  0.15s 1 result


In [30]:
A.show(results)



**tablet** *1*



## Search for spatial patterns
A few words on the construction of search templates.

The idea is that you mimick the things you are looking for
in your search template.
Embedded things are mimicked by indentation.

Let's search for a line with a case in it that is not further divided,
in which there is a numeral and an ideograph.

Here is our first attempt, and we show the first tablet only.
Note that you can have comments in a search template.
Lines that start with `#` are ignored.

In [31]:
query = '''
line
    case terminal=1
% order is not important
        sign type=ideograph
        sign type=numeral
'''
results = A.search(query)

  0.40s 10673 results


First a glance at the first 20 items in tabular view.

In [32]:
A.table(results, end=20)

n | p | tablet | sign | sign | sign | line | sign | case | sign | sign | case | sign | sign
--- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | ---
1| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">P448702</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">tablet P448702</a>|<span  class="hl hlup" >3(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >KASZ~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KASZ~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kasz-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >GI</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/GI.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-gi.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hldot" >line 2</span>|<span  class="hl hlup" >3(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 2b</span>|<span  class="hl hlup" >N</span> |<span  class="hl hlup" >2(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 2a</span>|<span  class="hl hlup" >KASZ~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KASZ~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kasz-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >NUN~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/NUN~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-nun-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
2| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P471695" title="to CDLI main page for this tablet" sec="P471695">P471695</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P471695" title="to CDLI main page for this tablet" sec="P471695">tablet P471695</a>|<span  class="hl" >case 2b1</span>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >ISZ~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/ISZ~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-isz-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 1a</span>|<span  class="hl hlup" >3(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >APIN~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/APIN~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-apin-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >3(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >UR4~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/UR4~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-ur4-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hldot" >line 1</span>|<span  class="hldot" >line 2</span>|<span  class="hldot" >line 1</span>|<span  class="hl hlup" >3(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >PAP~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/PAP~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-pap-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 1a</span>
3| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P482083" title="to CDLI main page for this tablet" sec="P482083">P482083</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P482083" title="to CDLI main page for this tablet" sec="P482083">tablet P482083</a>|<span  class="hl hlup" >1(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >SZE~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SZE~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-sze-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >N</span> |<span  class="hl hlup" >TAR~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/TAR~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-tar-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >3(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >SZE~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SZE~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-sze-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >KASZ~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KASZ~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kasz-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >U4</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/U4.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-u4.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >3(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N42~a)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N42~a).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n42-a).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N25)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N25).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n25).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >TAR~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/TAR~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-tar-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>|<span  class="hldot" >line 1</span>|<span  class="hldot" >line 2</span>
4| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000735" title="to CDLI main page for this tablet" sec="P000735">P000735</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000735" title="to CDLI main page for this tablet" sec="P000735">tablet P000735</a>|<span  class="hl hlup" >2(N39~a)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N39~a).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n39-a).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >HI@g~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/HI@g~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-hi(a)g-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >EN~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/EN~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-en-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hldot" >line 1</span>|<span  class="hl hlup" >SZIM~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SZIM~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-szim-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N24)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N24).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n24).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N28)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N28).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n28).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >SZE~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SZE~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-sze-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N30~c)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N30~c).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n30-c).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>
5| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000743" title="to CDLI main page for this tablet" sec="P000743">P000743</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000743" title="to CDLI main page for this tablet" sec="P000743">tablet P000743</a>|<span  class="hl hlup" >1(N28)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N28).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n28).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >HI@g~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/HI@g~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-hi(a)g-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hldot" >line 1</span>|<span  class="hl hlup" >MU</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/MU.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-mu.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N24)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N24).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n24).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N39~a)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N39~a).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n39-a).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N24)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N24).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n24).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >HI@g~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/HI@g~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-hi(a)g-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N39~a)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N39~a).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n39-a).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N28)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N28).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n28).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >ZATU714</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/ZATU714.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-zatu714.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 1a1</span>|<span  class="hl" >case 1a2</span>|<span  class="hl" >case 1b</span>|<span  class="hl hlup" >4(N39~a)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/4(N39~a).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-4(n39-a).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >ZATU714</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/ZATU714.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-zatu714.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
6| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000759" title="to CDLI main page for this tablet" sec="P000759">P000759</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000759" title="to CDLI main page for this tablet" sec="P000759">tablet P000759</a>|<span  class="hl" >case 1a2</span>|<span  class="hldot" >line 1</span>|<span  class="hl hlup" >2(N39~a)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N39~a).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n39-a).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N28)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N28).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n28).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >HI@g~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/HI@g~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-hi(a)g-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
7| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000760" title="to CDLI main page for this tablet" sec="P000760">P000760</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000760" title="to CDLI main page for this tablet" sec="P000760">tablet P000760</a>|<span  class="hl" >case 2a1</span>|<span  class="hl" >case 2a2</span>|<span  class="hl" >case 2c</span>|<span  class="hl" >case 3a1</span>|<span  class="hl" >case 3b</span>|<span  class="hl hlup" >2(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >SAL</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SAL.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-sal.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >KI</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KI.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-ki.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >SAL</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SAL.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-sal.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >SZA3~a1</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SZA3~a1.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-sza3-a1.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >8(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/8(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-8(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >U4</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/U4.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-u4.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >SZU2</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SZU2.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-szu2.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >SA~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SA~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-sa-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >ZATU759</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/ZATU759.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-zatu759.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >5(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/5(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-5(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >KISAL~b1</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KISAL~b1.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kisal-b1.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hldot" >line 2</span>|<span  class="hldot" >line 3</span>
8| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000783" title="to CDLI main page for this tablet" sec="P000783">P000783</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000783" title="to CDLI main page for this tablet" sec="P000783">tablet P000783</a>|<span  class="hl hlup" >1(N28)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N28).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n28).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hldot" >line 2</span>|<span  class="hl hlup" >HI</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/HI.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-hi.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 2a1</span>|<span  class="hl" >case 2a2</span>|<span  class="hl hlup" >4(N39~a)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/4(N39~a).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-4(n39-a).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >HI@g~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/HI@g~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-hi(a)g-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >3(N39~a)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N39~a).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n39-a).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N24)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N24).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n24).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
9| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001056" title="to CDLI main page for this tablet" sec="P001056">P001056</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001056" title="to CDLI main page for this tablet" sec="P001056">tablet P001056</a>|<span  class="hl hlup" >2(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >SZAGAN</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SZAGAN.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-szagan.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >EN~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/EN~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-en-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >NUN~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/NUN~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-nun-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >ERIM~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/ERIM~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-erim-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >KISAL~b1</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KISAL~b1.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kisal-b1.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >NAMESZDA</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/NAMESZDA.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-nameszda.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >KISAL~b1</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KISAL~b1.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kisal-b1.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >GESZTU~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/GESZTU~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-gesztu-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >TE</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/TE.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-te.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 3a</span>|<span  class="hl hlup" >AB~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/AB~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-ab-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 4a</span>|<span  class="hl hlup" >1(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 5a</span>|<span  class="hl hlup" >5(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/5(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-5(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 6a</span>|<span  class="hl hlup" >GURUSZ~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/GURUSZ~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-gurusz-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 1a</span>|<span  class="hl hlup" >UDU~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/UDU~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-udu-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 2a</span>|<span  class="hl hlup" >1(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >URI</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/URI.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-uri.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 3a</span>|<span  class="hl hlup" >ZATU752</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/ZATU752.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-zatu752.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >GAL~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/GAL~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-gal-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 4a</span>|<span  class="hl hlup" >EN~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/EN~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-en-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >NUNUZ~c</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/NUNUZ~c.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-nunuz-c.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 5a</span>|<span  class="hl hlup" >UMUN2</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/UMUN2.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-umun2.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >KU3~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KU3~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-ku3-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 6a</span>|<span  class="hl hlup" >TUR3~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/TUR3~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-tur3-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hldot" >line 3</span>|<span  class="hldot" >line 4</span>|<span  class="hldot" >line 5</span>|<span  class="hldot" >line 6</span>|<span  class="hldot" >line 1</span>|<span  class="hldot" >line 2</span>|<span  class="hldot" >line 3</span>|<span  class="hldot" >line 4</span>|<span  class="hldot" >line 5</span>|<span  class="hldot" >line 6</span>
10| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001709" title="to CDLI main page for this tablet" sec="P001709">P001709</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001709" title="to CDLI main page for this tablet" sec="P001709">tablet P001709</a>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b2</span>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b1</span>|<span  class="hl" >case 3b</span>|<span  class="hl hlup" >2(N51)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N51).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n51).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N34)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N34).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n34).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >6(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/6(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-6(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >GAR</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/GAR.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-gar.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >GUM~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/GUM~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-gum-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >4(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/4(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-4(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >DUG~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/DUG~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-dug-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >KASZ~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KASZ~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kasz-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 2b1</span>|<span  class="hl hlup" >1(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >DUG~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/DUG~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-dug-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >KASZ~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KASZ~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kasz-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >3(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >DUG~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/DUG~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-dug-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N08)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N08).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n08).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >DUG~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/DUG~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-dug-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >4(N34)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/4(N34).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-4(n34).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >4(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/4(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-4(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >DUG~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/DUG~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-dug-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >KASZ~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KASZ~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kasz-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N34)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N34).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n34).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >DUG~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/DUG~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-dug-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N34)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N34).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n34).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >4(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/4(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-4(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >KASZ~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KASZ~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kasz-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hldot" >line 1</span>|<span  class="hldot" >line 2</span>|<span  class="hldot" >line 3</span>|<span  class="hldot" >line 2</span>
11| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001712" title="to CDLI main page for this tablet" sec="P001712">P001712</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001712" title="to CDLI main page for this tablet" sec="P001712">tablet P001712</a>|<span  class="hldot" >line 3</span>|<span  class="hl" >case 3a</span>|<span  class="hl hlup" >3(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >5(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/5(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-5(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >GAL~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/GAL~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-gal-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >3(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 2a</span>|<span  class="hl hlup" >GESZTU~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/GESZTU~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-gesztu-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N34)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N34).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n34).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >NUN~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/NUN~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-nun-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >EN~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/EN~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-en-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 1a</span>|<span  class="hldot" >line 1</span>|<span  class="hldot" >line 2</span>
12| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001819" title="to CDLI main page for this tablet" sec="P001819">P001819</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001819" title="to CDLI main page for this tablet" sec="P001819">tablet P001819</a>|<span  class="hl hlup" >6(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/6(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-6(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >KI@n</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KI@n.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-ki(a)n.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >DUB~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/DUB~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-dub-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>|<span  class="hldot" >line 2</span>|<span  class="hl hlup" >4(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/4(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-4(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >NAM2</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/NAM2.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-nam2.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >3(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
13| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001951" title="to CDLI main page for this tablet" sec="P001951">P001951</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001951" title="to CDLI main page for this tablet" sec="P001951">tablet P001951</a>|<span  class="hldot" >line 4</span>|<span  class="hldot" >line 5</span>|<span  class="hl" >case 4a</span>|<span  class="hl" >case 4b</span>|<span  class="hl" >case 5a</span>|<span  class="hl" >case 5b</span>|<span  class="hl hlup" >1(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >NAGA~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/NAGA~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-naga-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >3(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >E2~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/E2~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-e2-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >ZATU659</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/ZATU659.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-zatu659.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >NIR~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/NIR~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-nir-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >SU~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SU~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-su-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >PAP~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/PAP~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-pap-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >3(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
14| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001971" title="to CDLI main page for this tablet" sec="P001971">P001971</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001971" title="to CDLI main page for this tablet" sec="P001971">tablet P001971</a>|<span  class="hldot" >line 1</span>|<span  class="hldot" >line 2</span>|<span  class="hldot" >line 3</span>|<span  class="hldot" >line 1</span>|<span  class="hldot" >line 2</span>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >DUG~c</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/DUG~c.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-dug-c.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >DUG~c</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/DUG~c.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-dug-c.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >GESZTU~c3</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/GESZTU~c3.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-gesztu-c3.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >KASZ~c</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KASZ~c.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kasz-c.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >3(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >DUG~c</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/DUG~c.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-dug-c.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >NUN~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/NUN~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-nun-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >KASZ~c</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KASZ~c.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kasz-c.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 1a</span>|<span  class="hl hlup" >GESZTU~c3</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/GESZTU~c3.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-gesztu-c3.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>|<span  class="hl hlup" >1(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >AN</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/AN.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-an.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >MUSZ3~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/MUSZ3~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-musz3-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 3c</span>|<span  class="hl" >case 1a</span>|<span  class="hl hlup" >KASZ~c</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KASZ~c.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kasz-c.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 1c</span>|<span  class="hl" >case 2a</span>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 2c</span>
15| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002002" title="to CDLI main page for this tablet" sec="P002002">P002002</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002002" title="to CDLI main page for this tablet" sec="P002002">tablet P002002</a>|<span  class="hldot" >line 6</span>|<span  class="hldot" >line 7</span>|<span  class="hldot" >line 8</span>|<span  class="hldot" >line 6</span>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>|<span  class="hl" >case 3a</span>|<span  class="hl" >case 3b</span>|<span  class="hl" >case 4a</span>|<span  class="hl" >case 5a</span>|<span  class="hldot" >line 1</span>|<span  class="hldot" >line 2</span>|<span  class="hldot" >line 3</span>|<span  class="hldot" >line 4</span>|<span  class="hldot" >line 5</span>|<span  class="hl" >case 5b</span>|<span  class="hldot" >line 6</span>|<span  class="hl" >case 8a</span>|<span  class="hldot" >line 9</span>|<span  class="hldot" >line 1</span>|<span  class="hldot" >line 2</span>|<span  class="hl" >case 8b</span>|<span  class="hl" >case 9a</span>|<span  class="hl" >case 1a</span>|<span  class="hldot" >line 7</span>|<span  class="hldot" >line 8</span>|<span  class="hl" >case 2a</span>|<span  class="hldot" >line 3</span>|<span  class="hl" >case 3a</span>|<span  class="hldot" >line 4</span>|<span  class="hl" >case 4a</span>|<span  class="hl" >case 4b</span>|<span  class="hldot" >line 5</span>|<span  class="hl" >case 5a</span>|<span  class="hl" >case 6a</span>|<span  class="hl" >case 7a</span>|<span  class="hl" >case 8a</span>|<span  class="hl" >case 8b</span>|<span  class="hldot" >line 3</span>|<span  class="hldot" >line 4</span>|<span  class="hldot" >line 5</span>|<span  class="hl" >case 3a</span>|<span  class="hl" >case 4a</span>|<span  class="hl" >case 5a1</span>|<span  class="hl" >case 5a2</span>|<span  class="hl" >case 6a</span>|<span  class="hl hlup" >3(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >KU6~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KU6~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-ku6-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >KISIM~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KISIM~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kisim-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >UNUG~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/UNUG~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-unug-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >SZEN~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SZEN~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-szen-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >PAP~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/PAP~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-pap-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >MUSZEN</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/MUSZEN.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-muszen.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >SZEN~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SZEN~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-szen-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >PAP~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/PAP~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-pap-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >SZU</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SZU.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-szu.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >PAP~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/PAP~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-pap-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >MUSZEN</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/MUSZEN.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-muszen.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >UKKIN~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/UKKIN~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-ukkin-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >SZEN~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SZEN~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-szen-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >PAP~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/PAP~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-pap-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >MUSZEN</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/MUSZEN.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-muszen.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >SZEN~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SZEN~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-szen-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >SI</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SI.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-si.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >RAD~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/RAD~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-rad-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >PAP~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/PAP~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-pap-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >MUSZEN</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/MUSZEN.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-muszen.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >DUG~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/DUG~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-dug-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >GA2~a2</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/GA2~a2.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-ga2-a2.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >GURUSZDA</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/GURUSZDA.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-guruszda.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >UNUG~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/UNUG~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-unug-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >5(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/5(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-5(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >SI</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SI.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-si.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >UKKIN~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/UKKIN~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-ukkin-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >SZE~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SZE~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-sze-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >3(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >E2~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/E2~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-e2-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >DUG~c</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/DUG~c.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-dug-c.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >GI</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/GI.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-gi.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >DUG~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/DUG~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-dug-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >KISIM~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KISIM~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kisim-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >KISIM~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KISIM~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kisim-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >PAP~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/PAP~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-pap-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >SZU</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SZU.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-szu.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >KISIM~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KISIM~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kisim-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >EN~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/EN~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-en-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >DU</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/DU.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-du.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >AB2</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/AB2.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-ab2.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >UKKIN~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/UKKIN~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-ukkin-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >SZU2</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SZU2.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-szu2.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >EN~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/EN~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-en-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >AN</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/AN.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-an.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >EN~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/EN~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-en-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >DU</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/DU.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-du.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >AB2</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/AB2.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-ab2.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >MUSZEN</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/MUSZEN.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-muszen.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >PAP~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/PAP~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-pap-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >KISIM~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KISIM~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kisim-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >U4</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/U4.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-u4.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >EN~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/EN~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-en-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >ME~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/ME~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-me-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >NUNUZ~a1</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/NUNUZ~a1.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-nunuz-a1.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >EN~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/EN~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-en-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >TI</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/TI.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-ti.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >UKKIN~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/UKKIN~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-ukkin-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >APIN~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/APIN~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-apin-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >ZATU752</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/ZATU752.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-zatu752.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >MUSZEN</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/MUSZEN.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-muszen.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >PAP~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/PAP~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-pap-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >4(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/4(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-4(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >4(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/4(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-4(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >DUG~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/DUG~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-dug-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >4(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/4(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-4(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >KISIM~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KISIM~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kisim-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >IM~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/IM~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-im-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >SZEN~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SZEN~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-szen-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N08)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N08).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n08).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >KISIM~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KISIM~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kisim-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >SZEN~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SZEN~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-szen-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >MAH~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/MAH~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-mah-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >DA~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/DA~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-da-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 6a</span>|<span  class="hl" >case 6b</span>|<span  class="hl" >case 7a</span>|<span  class="hl" >case 7b1</span>
16| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002012" title="to CDLI main page for this tablet" sec="P002012">P002012</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002012" title="to CDLI main page for this tablet" sec="P002012">tablet P002012</a>|<span  class="hldot" >line 1</span>|<span  class="hl" >case 1a</span>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >TUR</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/TUR.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-tur.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >UDU~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/UDU~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-udu-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >NUN~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/NUN~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-nun-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >BA</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/BA.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-ba.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
17| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002017" title="to CDLI main page for this tablet" sec="P002017">P002017</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002017" title="to CDLI main page for this tablet" sec="P002017">tablet P002017</a>|<span  class="hl hlup" >DUG~c</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/DUG~c.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-dug-c.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 2a2</span>|<span  class="hl hlup" >2(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >KASZ~d</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KASZ~d.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kasz-d.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >DUG~c</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/DUG~c.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-dug-c.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >KASZ~d</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KASZ~d.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kasz-d.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 1a1</span>|<span  class="hl" >case 1a2</span>|<span  class="hldot" >line 1</span>|<span  class="hl" >case 2a1</span>|<span  class="hldot" >line 2</span>
18| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002018" title="to CDLI main page for this tablet" sec="P002018">P002018</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002018" title="to CDLI main page for this tablet" sec="P002018">tablet P002018</a>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 2b1</span>|<span  class="hl hlup" >KASZ~d</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KASZ~d.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kasz-d.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >SILA3~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SILA3~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-sila3-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hldot" >line 2</span>|<span  class="hl" >case 2b2</span>|<span  class="hl hlup" >SILA3~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SILA3~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-sila3-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >HASZHUR</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/HASZHUR.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-haszhur.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
19| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002035" title="to CDLI main page for this tablet" sec="P002035">P002035</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002035" title="to CDLI main page for this tablet" sec="P002035">tablet P002035</a>|<span  class="hl hlup" >DUG~c</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/DUG~c.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-dug-c.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >NAGAR~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/NAGAR~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-nagar-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >NE~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/NE~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-ne-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >5(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/5(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-5(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >SZA@g</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SZA@g.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-sza(a)g.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >5(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/5(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-5(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hldot" >line 1</span>|<span  class="hl hlup" >SZA</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SZA.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-sza.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >HI@g~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/HI@g~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-hi(a)g-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b1</span>|<span  class="hl" >case 1b2</span>
20| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002068" title="to CDLI main page for this tablet" sec="P002068">P002068</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002068" title="to CDLI main page for this tablet" sec="P002068">tablet P002068</a>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b1</span>|<span  class="hl" >case 2b2</span>|<span  class="hl" >case 2a</span>|<span  class="hldot" >line 2</span>|<span  class="hldot" >line 2</span>|<span  class="hl hlup" >1(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >DUG~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/DUG~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-dug-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >DA~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/DA~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-da-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >GA2~c</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/GA2~c.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-ga2-c.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >SILA3~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SILA3~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-sila3-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >HI</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/HI.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-hi.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >SILA3~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SILA3~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-sila3-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >NI~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/NI~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-ni-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >BA</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/BA.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-ba.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >DUG~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/DUG~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-dug-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>

Ah, we were still in condensed mode.

For this query the table is more perspicuous in normal mode, so we tell not to condense.

In [33]:
A.table(results, condensed=False, end=20)

n | p | line | case | sign | sign
--- | --- | --- | --- | --- | ---
1| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">P448702</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702"><span  class="hldot" >line 2</span></a>|<span  class="hl" >case 2a</span>|<span  class="hl hlup" >KASZ~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KASZ~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kasz-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >N</span> 
2| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">P448702</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702"><span  class="hldot" >line 2</span></a>|<span  class="hl" >case 2a</span>|<span  class="hl hlup" >KASZ~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KASZ~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kasz-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
3| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">P448702</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702"><span  class="hldot" >line 2</span></a>|<span  class="hl" >case 2a</span>|<span  class="hl hlup" >KASZ~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KASZ~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kasz-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >3(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
4| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">P448702</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702"><span  class="hldot" >line 2</span></a>|<span  class="hl" >case 2a</span>|<span  class="hl hlup" >NUN~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/NUN~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-nun-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >N</span> 
5| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">P448702</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702"><span  class="hldot" >line 2</span></a>|<span  class="hl" >case 2a</span>|<span  class="hl hlup" >NUN~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/NUN~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-nun-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >2(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
6| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">P448702</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702"><span  class="hldot" >line 2</span></a>|<span  class="hl" >case 2a</span>|<span  class="hl hlup" >NUN~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/NUN~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-nun-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >3(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
7| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">P448702</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702"><span  class="hldot" >line 2</span></a>|<span  class="hl" >case 2b</span>|<span  class="hl hlup" >KASZ~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KASZ~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kasz-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >3(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
8| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">P448702</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702"><span  class="hldot" >line 2</span></a>|<span  class="hl" >case 2b</span>|<span  class="hl hlup" >GI</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/GI.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-gi.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >3(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
9| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P471695" title="to CDLI main page for this tablet" sec="P471695">P471695</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P471695" title="to CDLI main page for this tablet" sec="P471695"><span  class="hldot" >line 1</span></a>|<span  class="hl" >case 1a</span>|<span  class="hl hlup" >APIN~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/APIN~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-apin-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >3(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
10| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P471695" title="to CDLI main page for this tablet" sec="P471695">P471695</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P471695" title="to CDLI main page for this tablet" sec="P471695"><span  class="hldot" >line 1</span></a>|<span  class="hl" >case 1a</span>|<span  class="hl hlup" >APIN~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/APIN~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-apin-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >3(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
11| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P471695" title="to CDLI main page for this tablet" sec="P471695">P471695</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P471695" title="to CDLI main page for this tablet" sec="P471695"><span  class="hldot" >line 1</span></a>|<span  class="hl" >case 1a</span>|<span  class="hl hlup" >UR4~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/UR4~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-ur4-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >3(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
12| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P471695" title="to CDLI main page for this tablet" sec="P471695">P471695</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P471695" title="to CDLI main page for this tablet" sec="P471695"><span  class="hldot" >line 1</span></a>|<span  class="hl" >case 1a</span>|<span  class="hl hlup" >UR4~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/UR4~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-ur4-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >3(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
13| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P471695" title="to CDLI main page for this tablet" sec="P471695">P471695</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P471695" title="to CDLI main page for this tablet" sec="P471695"><span  class="hldot" >line 2</span></a>|<span  class="hl" >case 2b1</span>|<span  class="hl hlup" >PAP~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/PAP~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-pap-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >3(N57)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N57).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n57).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
14| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P471695" title="to CDLI main page for this tablet" sec="P471695">P471695</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P471695" title="to CDLI main page for this tablet" sec="P471695"><span  class="hldot" >line 1</span></a>|<span  class="hl" >case 1a</span>|<span  class="hl hlup" >ISZ~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/ISZ~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-isz-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
15| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P482083" title="to CDLI main page for this tablet" sec="P482083">P482083</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P482083" title="to CDLI main page for this tablet" sec="P482083"><span  class="hldot" >line 1</span></a>|<span  class="hl" >case 1a</span>|<span  class="hl hlup" >SZE~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SZE~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-sze-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >1(N14)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N14).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n14).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
16| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P482083" title="to CDLI main page for this tablet" sec="P482083">P482083</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P482083" title="to CDLI main page for this tablet" sec="P482083"><span  class="hldot" >line 1</span></a>|<span  class="hl" >case 1b</span>|<span  class="hl hlup" >TAR~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/TAR~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-tar-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >N</span> 
17| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P482083" title="to CDLI main page for this tablet" sec="P482083">P482083</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P482083" title="to CDLI main page for this tablet" sec="P482083"><span  class="hldot" >line 2</span></a>|<span  class="hl" >case 2a</span>|<span  class="hl hlup" >SZE~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SZE~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-sze-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >3(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
18| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P482083" title="to CDLI main page for this tablet" sec="P482083">P482083</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P482083" title="to CDLI main page for this tablet" sec="P482083"><span  class="hldot" >line 2</span></a>|<span  class="hl" >case 2a</span>|<span  class="hl hlup" >SZE~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SZE~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-sze-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >3(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
19| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P482083" title="to CDLI main page for this tablet" sec="P482083">P482083</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P482083" title="to CDLI main page for this tablet" sec="P482083"><span  class="hldot" >line 2</span></a>|<span  class="hl" >case 2a</span>|<span  class="hl hlup" >KASZ~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KASZ~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kasz-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >3(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
20| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P482083" title="to CDLI main page for this tablet" sec="P482083">P482083</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P482083" title="to CDLI main page for this tablet" sec="P482083"><span  class="hldot" >line 2</span></a>|<span  class="hl" >case 2a</span>|<span  class="hl hlup" >KASZ~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KASZ~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-kasz-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl hlup" >3(N01)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N01).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n01).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>

Now the results on the first tablet.

In [34]:
A.show(results, end=1)



**tablet** *1*



The order between the two signs is not defined by the template,
despite the fact that the line with the ideograph
precedes the line with the numeral.
Results may have the numeral and the ideograph in any order. 

In fact, the highlights above represent multiple results.
If a case has say 2 numerals and 3 ideographs, there are 6 possible
pairs.

By default, results are shown in *condensed* mode.
That means that results are shown per tablet, and on the result tablets
everything that is in some result is being highlighted.

It is also possible to see the uncondensed results.
That gives you an exact picture of each real result constellation.

In order to illustrate the difference, we focus on one tablet and one case.
This case has 3 numerals and 2 ideographs, so we expect 6 results.

In [35]:
query = '''
tablet catalogId=P448702
  line
    case terminal=1 number=2a
        sign type=ideograph
        sign type=numeral
'''
results = A.search(query)

  0.41s 6 results


We show them condensed (by default), so we expect 1 tablet with all ideographs and numerals in case `2a'` highlighted.

In [36]:
A.show(results, lineart=False)



**tablet** *1*



Now the same results in uncondensed mode. Expect 6 times the same tablet with
different highlighted pairs of signs.

Note that we can apply different highlight colors to different parts of the result.
The words in the pair are member 4 and 5.

The members that we do not map, will not be highlighted.
The members that we map to the empty string will be highlighted with the default color.

**NB:** Choose your colors from the
[CSS specification](https://developer.mozilla.org/en-US/docs/Web/CSS/color_value).

In [37]:
A.show(results, condensed=False, colorMap={3: '', 4: 'cyan', 5: 'magenta'}, lineart=False)



**tablet** *1*





**tablet** *2*





**tablet** *3*





**tablet** *4*





**tablet** *5*





**tablet** *6*



Color mapping works best for uncondensed results. If you condense results, some nodes may occupy
different positions in different results. It is unpredictable which color will be used 
for such nodes:

In [38]:
A.show(results, condensed=True, colorMap={3: '', 4: 'cyan', 5: 'magenta'}, lineart=False)



**tablet** *1*



You can enforce order.
We modify the template a little to state a
relational condition, namely that the ideograph follows the numeral.

In [40]:
query = '''
tablet catalogId=P448702
  line
    case terminal=1 number=2a
        sign type=ideograph
        > sign type=numeral
'''
results = A.search(query)
A.table(results, condensed=False, lineart=False)

  0.43s 6 results


n | p | tablet | line | case | sign | sign
--- | --- | --- | --- | --- | --- | ---
1| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">P448702</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">tablet P448702</a>|<span  class="hldot" >line 2</span>|<span  class="hl" >case 2a</span>|<span  class="hl hlup" >KASZ~b</span>|<span  class="hl hlup" >N</span>
2| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">P448702</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">tablet P448702</a>|<span  class="hldot" >line 2</span>|<span  class="hl" >case 2a</span>|<span  class="hl hlup" >KASZ~b</span>|<span  class="hl hlup" >2(N14)</span>
3| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">P448702</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">tablet P448702</a>|<span  class="hldot" >line 2</span>|<span  class="hl" >case 2a</span>|<span  class="hl hlup" >KASZ~b</span>|<span  class="hl hlup" >3(N01)</span>
4| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">P448702</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">tablet P448702</a>|<span  class="hldot" >line 2</span>|<span  class="hl" >case 2a</span>|<span  class="hl hlup" >NUN~a</span>|<span  class="hl hlup" >N</span>
5| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">P448702</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">tablet P448702</a>|<span  class="hldot" >line 2</span>|<span  class="hl" >case 2a</span>|<span  class="hl hlup" >NUN~a</span>|<span  class="hl hlup" >2(N14)</span>
6| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">P448702</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">tablet P448702</a>|<span  class="hldot" >line 2</span>|<span  class="hl" >case 2a</span>|<span  class="hl hlup" >NUN~a</span>|<span  class="hl hlup" >3(N01)</span>

Still six results.
No wonder, because the case has first three numerals in a row and then 2 ideographs.

Do you want the ideograph and the numeral to be *adjacent* as well?
We only have to add 1 character to the template to make it happen.

In [41]:
query = '''
tablet catalogId=P448702
  line
    case terminal=1 number=2a
        sign type=ideograph
        :> sign type=numeral
'''
results = A.search(query)

  0.41s 1 result


In [42]:
A.table(results, condensed=False, lineart=False)

n | p | tablet | line | case | sign | sign
--- | --- | --- | --- | --- | --- | ---
1| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">P448702</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">tablet P448702</a>|<span  class="hldot" >line 2</span>|<span  class="hl" >case 2a</span>|<span  class="hl hlup" >KASZ~b</span>|<span  class="hl hlup" >3(N01)</span>

In [43]:
A.show(results, colorMap={4: 'cyan', 5: 'magenta'}, lineart=False)



**tablet** *1*



By now it pays off to study the possibilities of
[search templates](https://dans-labs.github.io/text-fabric/Use/Search/#search-templates).

If you want a reminder of all possible spatial relationships between nodes, you can call it up
here in your notebook:

In [44]:
S.relationsLegend()

                      = left equal to right (as node)
                      # left unequal to right (as node)
                      < left before right (in canonical node ordering)
                      > left after right (in canonical node ordering)
                     == left occupies same slots as right
                     && left has overlapping slots with right
                     ## left and right do not have the same slot set
                     || left and right do not have common slots
                     [[ left embeds right
                     ]] left embedded in right
                     << left completely before right
                     >> left completely after right
                     =: left and right start at the same slot
                     := left and right end at the same slot
                     :: left and right start and end at the same slot
                     <: left immediately before right
                     :> left immediately after right
   

## Comparisons in templates: cases

Cases have a feature depth which indicate their nesting depth within a line.
It is not the depth *of* that case, but the depth *at* which that case occurs.

Comparison queries are handy to select cases of a certain minimum or maximum depth.

In [45]:
query = '''
case depth=3
'''
results = A.search(query)
A.table(results, end=20)

  0.01s 254 results


n | p | tablet | case | case
--- | --- | --- | --- | ---
1| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003357" title="to CDLI main page for this tablet" sec="P003357">P003357</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003357" title="to CDLI main page for this tablet" sec="P003357">tablet P003357</a>|<span  class="hl" >case 1b1B</span>|<span  class="hl" >case 1b1A</span>
2| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003537" title="to CDLI main page for this tablet" sec="P003537">P003537</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003537" title="to CDLI main page for this tablet" sec="P003537">tablet P003537</a>|<span  class="hl" >case 4b1A</span>|<span  class="hl" >case 4b1B</span>|<span  class="hl" >case 4b2A</span>|<span  class="hl" >case 4b2B</span>
3| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003589" title="to CDLI main page for this tablet" sec="P003589">P003589</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003589" title="to CDLI main page for this tablet" sec="P003589">tablet P003589</a>|<span  class="hl" >case 3b2A</span>|<span  class="hl" >case 3b2B</span>
4| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003822" title="to CDLI main page for this tablet" sec="P003822">P003822</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003822" title="to CDLI main page for this tablet" sec="P003822">tablet P003822</a>|<span  class="hl" >case 1b2A</span>|<span  class="hl" >case 1b2B</span>|<span  class="hl" >case 1a2A</span>|<span  class="hl" >case 1a2B</span>
5| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P004448" title="to CDLI main page for this tablet" sec="P004448">P004448</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P004448" title="to CDLI main page for this tablet" sec="P004448">tablet P004448</a>|<span  class="hl" >case 2b2A</span>|<span  class="hl" >case 2b2B</span>|<span  class="hl" >case 3a2A</span>|<span  class="hl" >case 3a2B</span>|<span  class="hl" >case 3b1A</span>|<span  class="hl" >case 3b1B</span>|<span  class="hl" >case 3b2A</span>|<span  class="hl" >case 3b2B</span>|<span  class="hl" >case 1a2A</span>|<span  class="hl" >case 1a2B</span>|<span  class="hl" >case 1b2A</span>|<span  class="hl" >case 1b2B</span>|<span  class="hl" >case 2a2A</span>|<span  class="hl" >case 2a2B</span>
6| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P004735" title="to CDLI main page for this tablet" sec="P004735">P004735</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P004735" title="to CDLI main page for this tablet" sec="P004735">tablet P004735</a>|<span  class="hl" >case 2b1A</span>|<span  class="hl" >case 2b1B</span>|<span  class="hl" >case 2b2A</span>|<span  class="hl" >case 2b2B</span>|<span  class="hl" >case 1b1A</span>|<span  class="hl" >case 1b1B</span>|<span  class="hl" >case 1b2A</span>|<span  class="hl" >case 1b2B</span>
7| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P218054" title="to CDLI main page for this tablet" sec="P218054">P218054</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P218054" title="to CDLI main page for this tablet" sec="P218054">tablet P218054</a>|<span  class="hl" >case 1a1A</span>|<span  class="hl" >case 1a1B</span>|<span  class="hl" >case 1a2A</span>|<span  class="hl" >case 1a2B</span>|<span  class="hl" >case 3a1A</span>|<span  class="hl" >case 3a1B</span>
8| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006380" title="to CDLI main page for this tablet" sec="P006380">P006380</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006380" title="to CDLI main page for this tablet" sec="P006380">tablet P006380</a>|<span  class="hl" >case 2c1a</span>|<span  class="hl" >case 2c1b</span>|<span  class="hl" >case 2c2a</span>|<span  class="hl" >case 2c2b</span>
9| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006378" title="to CDLI main page for this tablet" sec="P006378">P006378</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006378" title="to CDLI main page for this tablet" sec="P006378">tablet P006378</a>|<span  class="hl" >case 1b2a</span>|<span  class="hl" >case 1b2b</span>
10| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P325754" title="to CDLI main page for this tablet" sec="P325754">P325754</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P325754" title="to CDLI main page for this tablet" sec="P325754">tablet P325754</a>|<span  class="hl" >case 1c1a</span>|<span  class="hl" >case 1c1b</span>|<span  class="hl" >case 1c2a</span>|<span  class="hl" >case 1c2b</span>
11| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P325234" title="to CDLI main page for this tablet" sec="P325234">P325234</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P325234" title="to CDLI main page for this tablet" sec="P325234">tablet P325234</a>|<span  class="hl" >case 1c4a</span>|<span  class="hl" >case 1c4b</span>|<span  class="hl" >case 1c5a</span>|<span  class="hl" >case 1c6a</span>|<span  class="hl" >case 1c6b</span>|<span  class="hl" >case 1c7a</span>|<span  class="hl" >case 1c7b</span>|<span  class="hl" >case 1c1a</span>|<span  class="hl" >case 1c1b</span>
12| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P328730" title="to CDLI main page for this tablet" sec="P328730">P328730</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P328730" title="to CDLI main page for this tablet" sec="P328730">tablet P328730</a>|<span  class="hl" >case 2b1a</span>|<span  class="hl" >case 2b1b</span>|<span  class="hl" >case 2b2a</span>|<span  class="hl" >case 2b2b</span>
13| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P411608" title="to CDLI main page for this tablet" sec="P411608">P411608</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P411608" title="to CDLI main page for this tablet" sec="P411608">tablet P411608</a>|<span  class="hl" >case 1b2a</span>|<span  class="hl" >case 1b2b</span>
14| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P387752" title="to CDLI main page for this tablet" sec="P387752">P387752</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P387752" title="to CDLI main page for this tablet" sec="P387752">tablet P387752</a>|<span  class="hl" >case 1b1b</span>|<span  class="hl" >case 2b1a</span>|<span  class="hl" >case 2b1b</span>|<span  class="hl" >case 2b2a</span>|<span  class="hl" >case 2b2b</span>|<span  class="hl" >case 1b2a</span>|<span  class="hl" >case 1b2b</span>|<span  class="hl" >case 1b1a</span>
15| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006381" title="to CDLI main page for this tablet" sec="P006381">P006381</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006381" title="to CDLI main page for this tablet" sec="P006381">tablet P006381</a>|<span  class="hl" >case 1c2A</span>|<span  class="hl" >case 1c2B</span>|<span  class="hl" >case 1c1A</span>|<span  class="hl" >case 1c1B</span>
16| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006414" title="to CDLI main page for this tablet" sec="P006414">P006414</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006414" title="to CDLI main page for this tablet" sec="P006414">tablet P006414</a>|<span  class="hl" >case 2d2a</span>|<span  class="hl" >case 2d2b</span>|<span  class="hl" >case 1c2a</span>|<span  class="hl" >case 1c2b</span>
17| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002413" title="to CDLI main page for this tablet" sec="P002413">P002413</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002413" title="to CDLI main page for this tablet" sec="P002413">tablet P002413</a>|<span  class="hl" >case 1b1A</span>|<span  class="hl" >case 1b1B</span>|<span  class="hl" >case 1b1C</span>|<span  class="hl" >case 1b2A</span>|<span  class="hl" >case 1b2B</span>|<span  class="hl" >case 1b2C</span>
18| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002785" title="to CDLI main page for this tablet" sec="P002785">P002785</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002785" title="to CDLI main page for this tablet" sec="P002785">tablet P002785</a>|<span  class="hl" >case 2b2A</span>|<span  class="hl" >case 2b2B</span>
19| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002856" title="to CDLI main page for this tablet" sec="P002856">P002856</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002856" title="to CDLI main page for this tablet" sec="P002856">tablet P002856</a>|<span  class="hl" >case 3b1A</span>|<span  class="hl" >case 3b1B</span>
20| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003968" title="to CDLI main page for this tablet" sec="P003968">P003968</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003968" title="to CDLI main page for this tablet" sec="P003968">tablet P003968</a>|<span  class="hl" >case 1c3B</span>|<span  class="hl" >case 1c1A</span>|<span  class="hl" >case 1c1B</span>|<span  class="hl" >case 1c3A</span>

Are there deeper cases?

In [46]:
query = '''
case depth>3
'''
results = A.search(query)
A.table(results, end=20)

  0.01s 119 results


n | p | tablet | case | case | case | case | case | case
--- | --- | --- | --- | --- | --- | --- | --- | ---
1| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P004735" title="to CDLI main page for this tablet" sec="P004735">P004735</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P004735" title="to CDLI main page for this tablet" sec="P004735">tablet P004735</a>|<span  class="hl" >case 2b2B1</span>|<span  class="hl" >case 2b2B2</span>|<span  class="hl" >case 1b1B1</span>|<span  class="hl" >case 1b1B2</span>|<span  class="hl" >case 1b1B3</span>|<span  class="hl" >case 1b1B4</span>
2| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P218054" title="to CDLI main page for this tablet" sec="P218054">P218054</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P218054" title="to CDLI main page for this tablet" sec="P218054">tablet P218054</a>|<span  class="hl" >case 1a1A1</span>|<span  class="hl" >case 1a1A2</span>
3| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P325754" title="to CDLI main page for this tablet" sec="P325754">P325754</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P325754" title="to CDLI main page for this tablet" sec="P325754">tablet P325754</a>|<span  class="hl" >case 1c2b1</span>|<span  class="hl" >case 1c2b2</span>|<span  class="hl" >case 1c2b3</span>
4| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P325234" title="to CDLI main page for this tablet" sec="P325234">P325234</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P325234" title="to CDLI main page for this tablet" sec="P325234">tablet P325234</a>|<span  class="hl" >case 1c5a1</span>|<span  class="hl" >case 1c5a2</span>
5| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P411608" title="to CDLI main page for this tablet" sec="P411608">P411608</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P411608" title="to CDLI main page for this tablet" sec="P411608">tablet P411608</a>|<span  class="hl" >case 1b2b2</span>|<span  class="hl" >case 1b2a1</span>|<span  class="hl" >case 1b2a2</span>|<span  class="hl" >case 1b2b1</span>
6| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P387752" title="to CDLI main page for this tablet" sec="P387752">P387752</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P387752" title="to CDLI main page for this tablet" sec="P387752">tablet P387752</a>|<span  class="hl" >case 1b2b09</span>|<span  class="hl" >case 1b2b10</span>|<span  class="hl" >case 1b2b11</span>|<span  class="hl" >case 1b2b12</span>|<span  class="hl" >case 1b2b13</span>|<span  class="hl" >case 1b2b14</span>|<span  class="hl" >case 1b2b15</span>|<span  class="hl" >case 1b2b16</span>|<span  class="hl" >case 1b2b17</span>|<span  class="hl" >case 2b1b1</span>|<span  class="hl" >case 2b1b2</span>|<span  class="hl" >case 2b2b1</span>|<span  class="hl" >case 1b1b01</span>|<span  class="hl" >case 1b1b02</span>|<span  class="hl" >case 1b1b03</span>|<span  class="hl" >case 1b1b04</span>|<span  class="hl" >case 1b1b05</span>|<span  class="hl" >case 1b1b06</span>|<span  class="hl" >case 1b1b07</span>|<span  class="hl" >case 1b1b08</span>|<span  class="hl" >case 1b1b09</span>|<span  class="hl" >case 1b1b10</span>|<span  class="hl" >case 1b1b11</span>|<span  class="hl" >case 1b1b12</span>|<span  class="hl" >case 1b1b13</span>|<span  class="hl" >case 1b1b14</span>|<span  class="hl" >case 1b1b15</span>|<span  class="hl" >case 1b1b16</span>|<span  class="hl" >case 1b1b17</span>|<span  class="hl" >case 1b1b18</span>|<span  class="hl" >case 1b1b19</span>|<span  class="hl" >case 1b1b20</span>|<span  class="hl" >case 1b2b01</span>|<span  class="hl" >case 1b2b02</span>|<span  class="hl" >case 1b2b03</span>|<span  class="hl" >case 1b2b04</span>|<span  class="hl" >case 1b2b05</span>|<span  class="hl" >case 1b2b06</span>|<span  class="hl" >case 1b2b07</span>|<span  class="hl" >case 1b2b08</span>
7| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002856" title="to CDLI main page for this tablet" sec="P002856">P002856</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002856" title="to CDLI main page for this tablet" sec="P002856">tablet P002856</a>|<span  class="hl" >case 3b1B4</span>|<span  class="hl" >case 3b1B1</span>|<span  class="hl" >case 3b1B2</span>|<span  class="hl" >case 3b1B3</span>
8| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006036" title="to CDLI main page for this tablet" sec="P006036">P006036</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006036" title="to CDLI main page for this tablet" sec="P006036">tablet P006036</a>|<span  class="hl" >case 2b2B1</span>|<span  class="hl" >case 2b2B2</span>|<span  class="hl" >case 2b2B3</span>|<span  class="hl" >case 2b2B4</span>|<span  class="hl" >case 2b1B1</span>|<span  class="hl" >case 2b1B2</span>
9| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006056" title="to CDLI main page for this tablet" sec="P006056">P006056</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006056" title="to CDLI main page for this tablet" sec="P006056">tablet P006056</a>|<span  class="hl" >case 3b2B2</span>|<span  class="hl" >case 3b2B1</span>
10| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006092" title="to CDLI main page for this tablet" sec="P006092">P006092</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006092" title="to CDLI main page for this tablet" sec="P006092">tablet P006092</a>|<span  class="hl" >case 2b1A1</span>|<span  class="hl" >case 2b1A2</span>
11| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006160" title="to CDLI main page for this tablet" sec="P006160">P006160</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006160" title="to CDLI main page for this tablet" sec="P006160">tablet P006160</a>|<span  class="hl" >case 2b2B1</span>|<span  class="hl" >case 2b2B2</span>
12| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006295" title="to CDLI main page for this tablet" sec="P006295">P006295</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006295" title="to CDLI main page for this tablet" sec="P006295">tablet P006295</a>|<span  class="hl" >case 1a1B1</span>|<span  class="hl" >case 1a1B2</span>
13| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006307" title="to CDLI main page for this tablet" sec="P006307">P006307</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006307" title="to CDLI main page for this tablet" sec="P006307">tablet P006307</a>|<span  class="hl" >case 2b1B1</span>|<span  class="hl" >case 2b1B2</span>|<span  class="hl" >case 2b1B3</span>|<span  class="hl" >case 2b1B4</span>|<span  class="hl" >case 2b2B1</span>|<span  class="hl" >case 2b2B2</span>|<span  class="hl" >case 2b2B3</span>
14| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P005294" title="to CDLI main page for this tablet" sec="P005294">P005294</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P005294" title="to CDLI main page for this tablet" sec="P005294">tablet P005294</a>|<span  class="hl" >case 1b1B1</span>|<span  class="hl" >case 1b1B2</span>
15| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P005322" title="to CDLI main page for this tablet" sec="P005322">P005322</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P005322" title="to CDLI main page for this tablet" sec="P005322">tablet P005322</a>|<span  class="hl" >case 1b3B1</span>|<span  class="hl" >case 1b3B2</span>|<span  class="hl" >case 1b4B1</span>|<span  class="hl" >case 1b4B2</span>|<span  class="hl" >case 1b1B1</span>|<span  class="hl" >case 1b1B2</span>|<span  class="hl" >case 1b2B1</span>|<span  class="hl" >case 1b2B2</span>
16| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002694" title="to CDLI main page for this tablet" sec="P002694">P002694</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002694" title="to CDLI main page for this tablet" sec="P002694">tablet P002694</a>|<span  class="hl" >case 3b1B1</span>|<span  class="hl" >case 3b2B1</span>|<span  class="hl" >case 3b2B2</span>
17| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003529" title="to CDLI main page for this tablet" sec="P003529">P003529</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003529" title="to CDLI main page for this tablet" sec="P003529">tablet P003529</a>|<span  class="hl" >case 3b3B1</span>|<span  class="hl" >case 3b1A1</span>|<span  class="hl" >case 3b1A2</span>|<span  class="hl" >case 3b3B2</span>
18| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003531" title="to CDLI main page for this tablet" sec="P003531">P003531</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003531" title="to CDLI main page for this tablet" sec="P003531">tablet P003531</a>|<span  class="hl" >case 1b1A1</span>|<span  class="hl" >case 1b1A2</span>
19| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003808" title="to CDLI main page for this tablet" sec="P003808">P003808</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003808" title="to CDLI main page for this tablet" sec="P003808">tablet P003808</a>|<span  class="hl" >case 2b1B3</span>|<span  class="hl" >case 2b1B4</span>|<span  class="hl" >case 1b1B1</span>|<span  class="hl" >case 1b1B2</span>|<span  class="hl" >case 1b2B1</span>|<span  class="hl" >case 1b2B2</span>|<span  class="hl" >case 2b1B1</span>|<span  class="hl" >case 2b1B2</span>|<span  class="hl" >case 1b2B3</span>|<span  class="hl" >case 1b2B4</span>|<span  class="hl" >case 2b2B1</span>|<span  class="hl" >case 2b2B2</span>|<span  class="hl" >case 2b1B1</span>|<span  class="hl" >case 2a2A1</span>|<span  class="hl" >case 2a2A2</span>|<span  class="hl" >case 2b1B2</span>|<span  class="hl" >case 2b1B5</span>|<span  class="hl" >case 2b1B6</span>

Still deeper?

In [47]:
query = '''
case depth>4
'''
results = A.search(query)
A.table(results, end=20)

  0.01s 0 results


As a check: the cases with depth 4 should be exactly the cases with depth > 3:

In [48]:
query = '''
case depth=4
'''
results = A.search(query)
A.table(results, end=20)
tc4 = len(results)

  0.01s 119 results


n | p | tablet | case | case | case | case | case | case
--- | --- | --- | --- | --- | --- | --- | --- | ---
1| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P004735" title="to CDLI main page for this tablet" sec="P004735">P004735</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P004735" title="to CDLI main page for this tablet" sec="P004735">tablet P004735</a>|<span  class="hl" >case 2b2B1</span>|<span  class="hl" >case 2b2B2</span>|<span  class="hl" >case 1b1B1</span>|<span  class="hl" >case 1b1B2</span>|<span  class="hl" >case 1b1B3</span>|<span  class="hl" >case 1b1B4</span>
2| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P218054" title="to CDLI main page for this tablet" sec="P218054">P218054</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P218054" title="to CDLI main page for this tablet" sec="P218054">tablet P218054</a>|<span  class="hl" >case 1a1A1</span>|<span  class="hl" >case 1a1A2</span>
3| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P325754" title="to CDLI main page for this tablet" sec="P325754">P325754</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P325754" title="to CDLI main page for this tablet" sec="P325754">tablet P325754</a>|<span  class="hl" >case 1c2b1</span>|<span  class="hl" >case 1c2b2</span>|<span  class="hl" >case 1c2b3</span>
4| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P325234" title="to CDLI main page for this tablet" sec="P325234">P325234</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P325234" title="to CDLI main page for this tablet" sec="P325234">tablet P325234</a>|<span  class="hl" >case 1c5a1</span>|<span  class="hl" >case 1c5a2</span>
5| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P411608" title="to CDLI main page for this tablet" sec="P411608">P411608</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P411608" title="to CDLI main page for this tablet" sec="P411608">tablet P411608</a>|<span  class="hl" >case 1b2b2</span>|<span  class="hl" >case 1b2a1</span>|<span  class="hl" >case 1b2a2</span>|<span  class="hl" >case 1b2b1</span>
6| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P387752" title="to CDLI main page for this tablet" sec="P387752">P387752</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P387752" title="to CDLI main page for this tablet" sec="P387752">tablet P387752</a>|<span  class="hl" >case 1b2b09</span>|<span  class="hl" >case 1b2b10</span>|<span  class="hl" >case 1b2b11</span>|<span  class="hl" >case 1b2b12</span>|<span  class="hl" >case 1b2b13</span>|<span  class="hl" >case 1b2b14</span>|<span  class="hl" >case 1b2b15</span>|<span  class="hl" >case 1b2b16</span>|<span  class="hl" >case 1b2b17</span>|<span  class="hl" >case 2b1b1</span>|<span  class="hl" >case 2b1b2</span>|<span  class="hl" >case 2b2b1</span>|<span  class="hl" >case 1b1b01</span>|<span  class="hl" >case 1b1b02</span>|<span  class="hl" >case 1b1b03</span>|<span  class="hl" >case 1b1b04</span>|<span  class="hl" >case 1b1b05</span>|<span  class="hl" >case 1b1b06</span>|<span  class="hl" >case 1b1b07</span>|<span  class="hl" >case 1b1b08</span>|<span  class="hl" >case 1b1b09</span>|<span  class="hl" >case 1b1b10</span>|<span  class="hl" >case 1b1b11</span>|<span  class="hl" >case 1b1b12</span>|<span  class="hl" >case 1b1b13</span>|<span  class="hl" >case 1b1b14</span>|<span  class="hl" >case 1b1b15</span>|<span  class="hl" >case 1b1b16</span>|<span  class="hl" >case 1b1b17</span>|<span  class="hl" >case 1b1b18</span>|<span  class="hl" >case 1b1b19</span>|<span  class="hl" >case 1b1b20</span>|<span  class="hl" >case 1b2b01</span>|<span  class="hl" >case 1b2b02</span>|<span  class="hl" >case 1b2b03</span>|<span  class="hl" >case 1b2b04</span>|<span  class="hl" >case 1b2b05</span>|<span  class="hl" >case 1b2b06</span>|<span  class="hl" >case 1b2b07</span>|<span  class="hl" >case 1b2b08</span>
7| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002856" title="to CDLI main page for this tablet" sec="P002856">P002856</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002856" title="to CDLI main page for this tablet" sec="P002856">tablet P002856</a>|<span  class="hl" >case 3b1B4</span>|<span  class="hl" >case 3b1B1</span>|<span  class="hl" >case 3b1B2</span>|<span  class="hl" >case 3b1B3</span>
8| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006036" title="to CDLI main page for this tablet" sec="P006036">P006036</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006036" title="to CDLI main page for this tablet" sec="P006036">tablet P006036</a>|<span  class="hl" >case 2b2B1</span>|<span  class="hl" >case 2b2B2</span>|<span  class="hl" >case 2b2B3</span>|<span  class="hl" >case 2b2B4</span>|<span  class="hl" >case 2b1B1</span>|<span  class="hl" >case 2b1B2</span>
9| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006056" title="to CDLI main page for this tablet" sec="P006056">P006056</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006056" title="to CDLI main page for this tablet" sec="P006056">tablet P006056</a>|<span  class="hl" >case 3b2B2</span>|<span  class="hl" >case 3b2B1</span>
10| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006092" title="to CDLI main page for this tablet" sec="P006092">P006092</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006092" title="to CDLI main page for this tablet" sec="P006092">tablet P006092</a>|<span  class="hl" >case 2b1A1</span>|<span  class="hl" >case 2b1A2</span>
11| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006160" title="to CDLI main page for this tablet" sec="P006160">P006160</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006160" title="to CDLI main page for this tablet" sec="P006160">tablet P006160</a>|<span  class="hl" >case 2b2B1</span>|<span  class="hl" >case 2b2B2</span>
12| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006295" title="to CDLI main page for this tablet" sec="P006295">P006295</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006295" title="to CDLI main page for this tablet" sec="P006295">tablet P006295</a>|<span  class="hl" >case 1a1B1</span>|<span  class="hl" >case 1a1B2</span>
13| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006307" title="to CDLI main page for this tablet" sec="P006307">P006307</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006307" title="to CDLI main page for this tablet" sec="P006307">tablet P006307</a>|<span  class="hl" >case 2b1B1</span>|<span  class="hl" >case 2b1B2</span>|<span  class="hl" >case 2b1B3</span>|<span  class="hl" >case 2b1B4</span>|<span  class="hl" >case 2b2B1</span>|<span  class="hl" >case 2b2B2</span>|<span  class="hl" >case 2b2B3</span>
14| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P005294" title="to CDLI main page for this tablet" sec="P005294">P005294</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P005294" title="to CDLI main page for this tablet" sec="P005294">tablet P005294</a>|<span  class="hl" >case 1b1B1</span>|<span  class="hl" >case 1b1B2</span>
15| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P005322" title="to CDLI main page for this tablet" sec="P005322">P005322</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P005322" title="to CDLI main page for this tablet" sec="P005322">tablet P005322</a>|<span  class="hl" >case 1b3B1</span>|<span  class="hl" >case 1b3B2</span>|<span  class="hl" >case 1b4B1</span>|<span  class="hl" >case 1b4B2</span>|<span  class="hl" >case 1b1B1</span>|<span  class="hl" >case 1b1B2</span>|<span  class="hl" >case 1b2B1</span>|<span  class="hl" >case 1b2B2</span>
16| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002694" title="to CDLI main page for this tablet" sec="P002694">P002694</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002694" title="to CDLI main page for this tablet" sec="P002694">tablet P002694</a>|<span  class="hl" >case 3b1B1</span>|<span  class="hl" >case 3b2B1</span>|<span  class="hl" >case 3b2B2</span>
17| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003529" title="to CDLI main page for this tablet" sec="P003529">P003529</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003529" title="to CDLI main page for this tablet" sec="P003529">tablet P003529</a>|<span  class="hl" >case 3b3B1</span>|<span  class="hl" >case 3b1A1</span>|<span  class="hl" >case 3b1A2</span>|<span  class="hl" >case 3b3B2</span>
18| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003531" title="to CDLI main page for this tablet" sec="P003531">P003531</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003531" title="to CDLI main page for this tablet" sec="P003531">tablet P003531</a>|<span  class="hl" >case 1b1A1</span>|<span  class="hl" >case 1b1A2</span>
19| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003808" title="to CDLI main page for this tablet" sec="P003808">P003808</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003808" title="to CDLI main page for this tablet" sec="P003808">tablet P003808</a>|<span  class="hl" >case 2b1B3</span>|<span  class="hl" >case 2b1B4</span>|<span  class="hl" >case 1b1B1</span>|<span  class="hl" >case 1b1B2</span>|<span  class="hl" >case 1b2B1</span>|<span  class="hl" >case 1b2B2</span>|<span  class="hl" >case 2b1B1</span>|<span  class="hl" >case 2b1B2</span>|<span  class="hl" >case 1b2B3</span>|<span  class="hl" >case 1b2B4</span>|<span  class="hl" >case 2b2B1</span>|<span  class="hl" >case 2b2B2</span>|<span  class="hl" >case 2b1B1</span>|<span  class="hl" >case 2a2A1</span>|<span  class="hl" >case 2a2A2</span>|<span  class="hl" >case 2b1B2</span>|<span  class="hl" >case 2b1B5</span>|<span  class="hl" >case 2b1B6</span>

Terminal cases at depth 1 are top-level divisions of lines that are not themselves divided further.

In [49]:
query = '''
case depth=1 terminal
'''
results = A.search(query)
A.table(results, end=20)
tc1 = len(results)

  0.03s 5468 results


n | p | tablet | case | case
--- | --- | --- | --- | ---
1| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">P448702</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">tablet P448702</a>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>
2| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P471695" title="to CDLI main page for this tablet" sec="P471695">P471695</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P471695" title="to CDLI main page for this tablet" sec="P471695">tablet P471695</a>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 1a</span>
3| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P482083" title="to CDLI main page for this tablet" sec="P482083">P482083</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P482083" title="to CDLI main page for this tablet" sec="P482083">tablet P482083</a>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>
4| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006438" title="to CDLI main page for this tablet" sec="P006438">P006438</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006438" title="to CDLI main page for this tablet" sec="P006438">tablet P006438</a>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>|<span  class="hl" >case 2c</span>|<span  class="hl" >case 2d</span>
5| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000735" title="to CDLI main page for this tablet" sec="P000735">P000735</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000735" title="to CDLI main page for this tablet" sec="P000735">tablet P000735</a>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>
6| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000743" title="to CDLI main page for this tablet" sec="P000743">P000743</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000743" title="to CDLI main page for this tablet" sec="P000743">tablet P000743</a>|<span  class="hl" >case 1b</span>
7| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000759" title="to CDLI main page for this tablet" sec="P000759">P000759</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000759" title="to CDLI main page for this tablet" sec="P000759">tablet P000759</a>|<span  class="hl" >case 1b</span>
8| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000760" title="to CDLI main page for this tablet" sec="P000760">P000760</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000760" title="to CDLI main page for this tablet" sec="P000760">tablet P000760</a>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>|<span  class="hl" >case 2c</span>|<span  class="hl" >case 3b</span>
9| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000783" title="to CDLI main page for this tablet" sec="P000783">P000783</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000783" title="to CDLI main page for this tablet" sec="P000783">tablet P000783</a>|<span  class="hl" >case 2b</span>
10| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001056" title="to CDLI main page for this tablet" sec="P001056">P001056</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001056" title="to CDLI main page for this tablet" sec="P001056">tablet P001056</a>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>|<span  class="hl" >case 3a</span>|<span  class="hl" >case 3b</span>|<span  class="hl" >case 4a</span>|<span  class="hl" >case 4b</span>|<span  class="hl" >case 5a</span>|<span  class="hl" >case 5b</span>|<span  class="hl" >case 6a</span>|<span  class="hl" >case 6b</span>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>|<span  class="hl" >case 3a</span>|<span  class="hl" >case 3b</span>|<span  class="hl" >case 4a</span>|<span  class="hl" >case 4b</span>|<span  class="hl" >case 5a</span>|<span  class="hl" >case 5b</span>|<span  class="hl" >case 6a</span>|<span  class="hl" >case 6b</span>
11| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001705" title="to CDLI main page for this tablet" sec="P001705">P001705</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001705" title="to CDLI main page for this tablet" sec="P001705">tablet P001705</a>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>|<span  class="hl" >case 3a</span>|<span  class="hl" >case 3b</span>
12| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001709" title="to CDLI main page for this tablet" sec="P001709">P001709</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001709" title="to CDLI main page for this tablet" sec="P001709">tablet P001709</a>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 3a</span>|<span  class="hl" >case 3b</span>|<span  class="hl" >case 2a</span>
13| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001712" title="to CDLI main page for this tablet" sec="P001712">P001712</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001712" title="to CDLI main page for this tablet" sec="P001712">tablet P001712</a>|<span  class="hl" >case 2b</span>|<span  class="hl" >case 3a</span>|<span  class="hl" >case 3b</span>|<span  class="hl" >case 4a</span>|<span  class="hl" >case 4b</span>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>|<span  class="hl" >case 2a</span>
14| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001819" title="to CDLI main page for this tablet" sec="P001819">P001819</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001819" title="to CDLI main page for this tablet" sec="P001819">tablet P001819</a>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>
15| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001903" title="to CDLI main page for this tablet" sec="P001903">P001903</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001903" title="to CDLI main page for this tablet" sec="P001903">tablet P001903</a>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>|<span  class="hl" >case 3a</span>|<span  class="hl" >case 3b</span>
16| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001951" title="to CDLI main page for this tablet" sec="P001951">P001951</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001951" title="to CDLI main page for this tablet" sec="P001951">tablet P001951</a>|<span  class="hl" >case 5a</span>|<span  class="hl" >case 5b</span>|<span  class="hl" >case 4a</span>|<span  class="hl" >case 4b</span>
17| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001971" title="to CDLI main page for this tablet" sec="P001971">P001971</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001971" title="to CDLI main page for this tablet" sec="P001971">tablet P001971</a>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>|<span  class="hl" >case 2c</span>|<span  class="hl" >case 3a</span>|<span  class="hl" >case 3b</span>|<span  class="hl" >case 3c</span>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>|<span  class="hl" >case 1c</span>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>|<span  class="hl" >case 2c</span>
18| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002002" title="to CDLI main page for this tablet" sec="P002002">P002002</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002002" title="to CDLI main page for this tablet" sec="P002002">tablet P002002</a>|<span  class="hl" >case 7a</span>|<span  class="hl" >case 7b</span>|<span  class="hl" >case 8a</span>|<span  class="hl" >case 8b</span>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>|<span  class="hl" >case 3a</span>|<span  class="hl" >case 3b</span>|<span  class="hl" >case 4a</span>|<span  class="hl" >case 4b</span>|<span  class="hl" >case 5b</span>|<span  class="hl" >case 6a</span>|<span  class="hl" >case 6b</span>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>|<span  class="hl" >case 3a</span>|<span  class="hl" >case 3b</span>|<span  class="hl" >case 4a</span>|<span  class="hl" >case 4b</span>|<span  class="hl" >case 5a</span>|<span  class="hl" >case 5b</span>|<span  class="hl" >case 6a</span>|<span  class="hl" >case 6b</span>|<span  class="hl" >case 7a</span>|<span  class="hl" >case 8a</span>|<span  class="hl" >case 8b</span>|<span  class="hl" >case 9a</span>|<span  class="hl" >case 9b</span>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>|<span  class="hl" >case 3a</span>|<span  class="hl" >case 3b</span>|<span  class="hl" >case 4a</span>|<span  class="hl" >case 4b</span>|<span  class="hl" >case 5a</span>|<span  class="hl" >case 5b</span>|<span  class="hl" >case 6a</span>|<span  class="hl" >case 6b</span>
19| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002012" title="to CDLI main page for this tablet" sec="P002012">P002012</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002012" title="to CDLI main page for this tablet" sec="P002012">tablet P002012</a>|<span  class="hl" >case 2b</span>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 2a</span>
20| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002017" title="to CDLI main page for this tablet" sec="P002017">P002017</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002017" title="to CDLI main page for this tablet" sec="P002017">tablet P002017</a>|<span  class="hl" >case 2b</span>|<span  class="hl" >case 1b</span>

Now let us select both the terminal cases of level 1 and 4.
They are disjunct, so the amounts should add up.

In [50]:
query = '''
case depth=1|4 terminal
'''
results = A.search(query)
A.table(results, end=20)
tc14 = len(results)
print(f'{tc1} + {tc4} = {tc1 + tc4} = {tc14}')

  0.03s 5587 results


n | p | tablet | case | case
--- | --- | --- | --- | ---
1| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">P448702</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">tablet P448702</a>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>
2| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P471695" title="to CDLI main page for this tablet" sec="P471695">P471695</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P471695" title="to CDLI main page for this tablet" sec="P471695">tablet P471695</a>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 1a</span>
3| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P482083" title="to CDLI main page for this tablet" sec="P482083">P482083</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P482083" title="to CDLI main page for this tablet" sec="P482083">tablet P482083</a>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>
4| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006438" title="to CDLI main page for this tablet" sec="P006438">P006438</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006438" title="to CDLI main page for this tablet" sec="P006438">tablet P006438</a>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>|<span  class="hl" >case 2c</span>|<span  class="hl" >case 2d</span>
5| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000735" title="to CDLI main page for this tablet" sec="P000735">P000735</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000735" title="to CDLI main page for this tablet" sec="P000735">tablet P000735</a>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>
6| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000743" title="to CDLI main page for this tablet" sec="P000743">P000743</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000743" title="to CDLI main page for this tablet" sec="P000743">tablet P000743</a>|<span  class="hl" >case 1b</span>
7| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000759" title="to CDLI main page for this tablet" sec="P000759">P000759</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000759" title="to CDLI main page for this tablet" sec="P000759">tablet P000759</a>|<span  class="hl" >case 1b</span>
8| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000760" title="to CDLI main page for this tablet" sec="P000760">P000760</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000760" title="to CDLI main page for this tablet" sec="P000760">tablet P000760</a>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>|<span  class="hl" >case 2c</span>|<span  class="hl" >case 3b</span>
9| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000783" title="to CDLI main page for this tablet" sec="P000783">P000783</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P000783" title="to CDLI main page for this tablet" sec="P000783">tablet P000783</a>|<span  class="hl" >case 2b</span>
10| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001056" title="to CDLI main page for this tablet" sec="P001056">P001056</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001056" title="to CDLI main page for this tablet" sec="P001056">tablet P001056</a>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>|<span  class="hl" >case 3a</span>|<span  class="hl" >case 3b</span>|<span  class="hl" >case 4a</span>|<span  class="hl" >case 4b</span>|<span  class="hl" >case 5a</span>|<span  class="hl" >case 5b</span>|<span  class="hl" >case 6a</span>|<span  class="hl" >case 6b</span>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>|<span  class="hl" >case 3a</span>|<span  class="hl" >case 3b</span>|<span  class="hl" >case 4a</span>|<span  class="hl" >case 4b</span>|<span  class="hl" >case 5a</span>|<span  class="hl" >case 5b</span>|<span  class="hl" >case 6a</span>|<span  class="hl" >case 6b</span>
11| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001705" title="to CDLI main page for this tablet" sec="P001705">P001705</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001705" title="to CDLI main page for this tablet" sec="P001705">tablet P001705</a>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>|<span  class="hl" >case 3a</span>|<span  class="hl" >case 3b</span>
12| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001709" title="to CDLI main page for this tablet" sec="P001709">P001709</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001709" title="to CDLI main page for this tablet" sec="P001709">tablet P001709</a>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 3a</span>|<span  class="hl" >case 3b</span>|<span  class="hl" >case 2a</span>
13| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001712" title="to CDLI main page for this tablet" sec="P001712">P001712</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001712" title="to CDLI main page for this tablet" sec="P001712">tablet P001712</a>|<span  class="hl" >case 2b</span>|<span  class="hl" >case 3a</span>|<span  class="hl" >case 3b</span>|<span  class="hl" >case 4a</span>|<span  class="hl" >case 4b</span>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>|<span  class="hl" >case 2a</span>
14| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001819" title="to CDLI main page for this tablet" sec="P001819">P001819</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001819" title="to CDLI main page for this tablet" sec="P001819">tablet P001819</a>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>
15| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001903" title="to CDLI main page for this tablet" sec="P001903">P001903</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001903" title="to CDLI main page for this tablet" sec="P001903">tablet P001903</a>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>|<span  class="hl" >case 3a</span>|<span  class="hl" >case 3b</span>
16| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001951" title="to CDLI main page for this tablet" sec="P001951">P001951</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001951" title="to CDLI main page for this tablet" sec="P001951">tablet P001951</a>|<span  class="hl" >case 5a</span>|<span  class="hl" >case 5b</span>|<span  class="hl" >case 4a</span>|<span  class="hl" >case 4b</span>
17| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001971" title="to CDLI main page for this tablet" sec="P001971">P001971</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P001971" title="to CDLI main page for this tablet" sec="P001971">tablet P001971</a>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>|<span  class="hl" >case 2c</span>|<span  class="hl" >case 3a</span>|<span  class="hl" >case 3b</span>|<span  class="hl" >case 3c</span>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>|<span  class="hl" >case 1c</span>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>|<span  class="hl" >case 2c</span>
18| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002002" title="to CDLI main page for this tablet" sec="P002002">P002002</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002002" title="to CDLI main page for this tablet" sec="P002002">tablet P002002</a>|<span  class="hl" >case 7a</span>|<span  class="hl" >case 7b</span>|<span  class="hl" >case 8a</span>|<span  class="hl" >case 8b</span>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>|<span  class="hl" >case 3a</span>|<span  class="hl" >case 3b</span>|<span  class="hl" >case 4a</span>|<span  class="hl" >case 4b</span>|<span  class="hl" >case 5b</span>|<span  class="hl" >case 6a</span>|<span  class="hl" >case 6b</span>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>|<span  class="hl" >case 3a</span>|<span  class="hl" >case 3b</span>|<span  class="hl" >case 4a</span>|<span  class="hl" >case 4b</span>|<span  class="hl" >case 5a</span>|<span  class="hl" >case 5b</span>|<span  class="hl" >case 6a</span>|<span  class="hl" >case 6b</span>|<span  class="hl" >case 7a</span>|<span  class="hl" >case 8a</span>|<span  class="hl" >case 8b</span>|<span  class="hl" >case 9a</span>|<span  class="hl" >case 9b</span>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 1b</span>|<span  class="hl" >case 2a</span>|<span  class="hl" >case 2b</span>|<span  class="hl" >case 3a</span>|<span  class="hl" >case 3b</span>|<span  class="hl" >case 4a</span>|<span  class="hl" >case 4b</span>|<span  class="hl" >case 5a</span>|<span  class="hl" >case 5b</span>|<span  class="hl" >case 6a</span>|<span  class="hl" >case 6b</span>
19| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002012" title="to CDLI main page for this tablet" sec="P002012">P002012</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002012" title="to CDLI main page for this tablet" sec="P002012">tablet P002012</a>|<span  class="hl" >case 2b</span>|<span  class="hl" >case 1a</span>|<span  class="hl" >case 2a</span>
20| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002017" title="to CDLI main page for this tablet" sec="P002017">P002017</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002017" title="to CDLI main page for this tablet" sec="P002017">tablet P002017</a>|<span  class="hl" >case 2b</span>|<span  class="hl" >case 1b</span>

5468 + 119 = 5587 = 5587


## Relational patterns: quads

Quads are compositions of signs by means of *operators*, such as `.` and `x`.
The operators are coded as an *edge* feature with values. The `op`-edges are between the signs/quads that are combined,
and the values of the `op` edges are the names of the operators in question.

Which operators do we have?

In [51]:
for (op, freq) in E.op.freqList():
    print(f'{op} : {freq:>5}x')

x :  2346x
. :  1042x
& :   222x
+ :   200x


Between how many sign pairs do we have an operator?

In [52]:
query = '''
sign
-op> sign
'''
results = A.search(query)

  0.24s 3642 results


Lets specifically ask for the `x` operator:

In [53]:
query = '''
sign
-op=x> sign
'''
results = A.search(query)

  0.34s 2238 results


Less than expected?

We must not forget the combinations between quads and between quads and signs.

We write a function that gives all pairs of sign/quads connected by a specific operator.

This is a fine illustration of how you can use programming to compose search templates,
instead of writing them out yourself.

In [54]:
def getCombi(op):
    types = ('sign', 'quad')
    allResults = []
    for type1 in types:
        for type2 in types:
            query = f'''
{type1}
-op{op}> {type2}
'''
            results = A.search(query, silent=True)
            print(f'{len(results):>5} {type1} {op} {type2}')
            allResults += results
    print(f'{len(allResults):>5} {op}')

Now we can count all combinations with `x`:

In [55]:
getCombi('=x')

 2238 sign =x sign
  105 sign =x quad
    3 quad =x sign
    0 quad =x quad
 2346 =x


In [56]:
getCombi('=.')

  985 sign =. sign
   43 sign =. quad
   14 quad =. sign
    0 quad =. quad
 1042 =.


In [57]:
getCombi('=&')

  220 sign =& sign
    1 sign =& quad
    0 quad =& sign
    1 quad =& quad
  222 =&


In [58]:
getCombi('=+')

  199 sign =+ sign
    0 sign =+ quad
    0 quad =+ sign
    1 quad =+ quad
  200 =+


In exact agreement with the results of `E.op.freqList()` above.
But we are more flexible!

We can ask for more operators at the same time.

In [59]:
getCombi('=x|+')

 2437 sign =x|+ sign
  105 sign =x|+ quad
    3 quad =x|+ sign
    1 quad =x|+ quad
 2546 =x|+


In [60]:
getCombi('~[^a-z]')

 1404 sign ~[^a-z] sign
   44 sign ~[^a-z] quad
   14 quad ~[^a-z] sign
    2 quad ~[^a-z] quad
 1464 ~[^a-z]


Finally, we zoom in on the rare cases where the operator is `x` used between a quad and a sign.
We want to see the show the lines where they occur.

In [61]:
query = '''
line
  quad
  -op=x> sign
'''
results = A.search(query)
A.show(results, withNodes=True)

  0.04s 3 results




**tablet** *1*





**tablet** *2*





**tablet** *3*



## Regular expressions in templates
We can use regular expressions in our search templates.

### Digits in graphemes
We search for non-numeral signs whose graphemes contains digits.

In [62]:
query = '''
sign type=ideograph grapheme~[0-9]
'''
results = A.search(query)
A.table(results, withNodes=True, end=5)

  0.25s 14558 results


n | p | tablet | sign | sign
--- | --- | --- | --- | ---
1| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">P448702</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">tablet P448702</a> *143892* |<span  class="hl" >U4</span> *75*  <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/U4.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-u4.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >U4</span> *76*  <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/U4.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-u4.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
2| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448703" title="to CDLI main page for this tablet" sec="P448703">P448703</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448703" title="to CDLI main page for this tablet" sec="P448703">tablet P448703</a> *143893* |<span  class="hl" >U4</span> *97*  <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/U4.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-u4.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >U4</span> *100*  <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/U4.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-u4.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >U4</span> *87*  <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/U4.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-u4.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >U4</span> *90*  <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/U4.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-u4.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >U4</span> *93*  <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/U4.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-u4.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
3| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P471695" title="to CDLI main page for this tablet" sec="P471695">P471695</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P471695" title="to CDLI main page for this tablet" sec="P471695">tablet P471695</a> *143894* |<span  class="hl" >ZATU759</span> *114*  <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/ZATU759.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-zatu759.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >GIR3~c</span> *140*  <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/GIR3~c.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-gir3-c.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >UR4~a</span> *111*  <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/UR4~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-ur4-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
4| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P482082" title="to CDLI main page for this tablet" sec="P482082">P482082</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P482082" title="to CDLI main page for this tablet" sec="P482082">tablet P482082</a> *143895* |<span  class="hl" >ZATU694~c</span> *155*  <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/ZATU694~c.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-zatu694-c.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
5| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P482083" title="to CDLI main page for this tablet" sec="P482083">P482083</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P482083" title="to CDLI main page for this tablet" sec="P482083">tablet P482083</a> *143896* |<span  class="hl" >U4</span> *169*  <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/U4.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-u4.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>

We can add a bit more context easily:

In [63]:
query = '''
tablet
  face
    column
      line
        sign type=ideograph grapheme~[0-9]
'''
results = A.search(query)
A.table(results, condensed=False, end=20)

  0.40s 14558 results


n | p | tablet | face | column | line | sign
--- | --- | --- | --- | --- | --- | ---
1| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">P448702</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">tablet P448702</a>|<span  class="hl" >face obverse</span>|<span  class="hldot hlup" >column 2</span>|<span  class="hldot hlup" >line 1</span>|<span  class="hl hlup" >U4</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/U4.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-u4.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
2| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">P448702</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448702" title="to CDLI main page for this tablet" sec="P448702">tablet P448702</a>|<span  class="hl" >face obverse</span>|<span  class="hldot hlup" >column 2</span>|<span  class="hldot hlup" >line 1</span>|<span  class="hl hlup" >U4</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/U4.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-u4.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
3| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448703" title="to CDLI main page for this tablet" sec="P448703">P448703</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448703" title="to CDLI main page for this tablet" sec="P448703">tablet P448703</a>|<span  class="hl" >face obverse</span>|<span  class="hldot hlup" >column 1</span>|<span  class="hldot hlup" >line 1</span>|<span  class="hl hlup" >U4</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/U4.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-u4.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
4| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448703" title="to CDLI main page for this tablet" sec="P448703">P448703</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448703" title="to CDLI main page for this tablet" sec="P448703">tablet P448703</a>|<span  class="hl" >face obverse</span>|<span  class="hldot hlup" >column 1</span>|<span  class="hldot hlup" >line 2</span>|<span  class="hl hlup" >U4</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/U4.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-u4.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
5| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448703" title="to CDLI main page for this tablet" sec="P448703">P448703</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448703" title="to CDLI main page for this tablet" sec="P448703">tablet P448703</a>|<span  class="hl" >face obverse</span>|<span  class="hldot hlup" >column 1</span>|<span  class="hldot hlup" >line 3</span>|<span  class="hl hlup" >U4</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/U4.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-u4.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
6| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448703" title="to CDLI main page for this tablet" sec="P448703">P448703</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448703" title="to CDLI main page for this tablet" sec="P448703">tablet P448703</a>|<span  class="hl" >face obverse</span>|<span  class="hldot hlup" >column 1</span>|<span  class="hldot hlup" >line 4</span>|<span  class="hl hlup" >U4</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/U4.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-u4.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
7| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448703" title="to CDLI main page for this tablet" sec="P448703">P448703</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448703" title="to CDLI main page for this tablet" sec="P448703">tablet P448703</a>|<span  class="hl" >face obverse</span>|<span  class="hldot hlup" >column 1</span>|<span  class="hldot hlup" >line 5</span>|<span  class="hl hlup" >U4</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/U4.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-u4.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
8| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P471695" title="to CDLI main page for this tablet" sec="P471695">P471695</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P471695" title="to CDLI main page for this tablet" sec="P471695">tablet P471695</a>|<span  class="hl" >face obverse</span>|<span  class="hldot hlup" >column 1</span>|<span  class="hldot hlup" >line 1</span>|<span  class="hl hlup" >UR4~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/UR4~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-ur4-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
9| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P471695" title="to CDLI main page for this tablet" sec="P471695">P471695</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P471695" title="to CDLI main page for this tablet" sec="P471695">tablet P471695</a>|<span  class="hl" >face obverse</span>|<span  class="hldot hlup" >column 1</span>|<span  class="hldot hlup" >line 1</span>|<span  class="hl hlup" >ZATU759</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/ZATU759.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-zatu759.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
10| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P471695" title="to CDLI main page for this tablet" sec="P471695">P471695</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P471695" title="to CDLI main page for this tablet" sec="P471695">tablet P471695</a>|<span  class="hl" >face obverse</span>|<span  class="hldot hlup" >column 2</span>|<span  class="hldot hlup" >line 1</span>|<span  class="hl hlup" >GIR3~c</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/GIR3~c.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-gir3-c.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
11| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P482082" title="to CDLI main page for this tablet" sec="P482082">P482082</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P482082" title="to CDLI main page for this tablet" sec="P482082">tablet P482082</a>|<span  class="hl" >face obverse</span>|<span  class="hldot hlup" >column 1</span>|<span  class="hldot hlup" >line 2</span>|<span  class="hl hlup" >ZATU694~c</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/ZATU694~c.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-zatu694-c.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
12| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P482083" title="to CDLI main page for this tablet" sec="P482083">P482083</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P482083" title="to CDLI main page for this tablet" sec="P482083">tablet P482083</a>|<span  class="hl" >face obverse</span>|<span  class="hldot hlup" >column 1</span>|<span  class="hldot hlup" >line 2</span>|<span  class="hl hlup" >U4</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/U4.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-u4.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
13| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P499393" title="to CDLI main page for this tablet" sec="P499393">P499393</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P499393" title="to CDLI main page for this tablet" sec="P499393">tablet P499393</a>|<span  class="hl" >face obverse</span>|<span  class="hldot hlup" >column 1</span>|<span  class="hldot hlup" >line 1</span>|<span  class="hl hlup" >U2~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/U2~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-u2-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
14| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P499393" title="to CDLI main page for this tablet" sec="P499393">P499393</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P499393" title="to CDLI main page for this tablet" sec="P499393">tablet P499393</a>|<span  class="hl" >face obverse</span>|<span  class="hldot hlup" >column 1</span>|<span  class="hldot hlup" >line 2</span>|<span  class="hl hlup" >U2~b</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/U2~b.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-u2-b.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
15| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P499393" title="to CDLI main page for this tablet" sec="P499393">P499393</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P499393" title="to CDLI main page for this tablet" sec="P499393">tablet P499393</a>|<span  class="hl" >face obverse</span>|<span  class="hldot hlup" >column 1</span>|<span  class="hldot hlup" >line 5</span>|<span  class="hl hlup" >SIG2~a1</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SIG2~a1.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-sig2-a1.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
16| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P499393" title="to CDLI main page for this tablet" sec="P499393">P499393</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P499393" title="to CDLI main page for this tablet" sec="P499393">tablet P499393</a>|<span  class="hl" >face obverse</span>|<span  class="hldot hlup" >column 2</span>|<span  class="hldot hlup" >line 4</span>|<span  class="hl hlup" >SZU2</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/SZU2.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-szu2.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
17| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P504412" title="to CDLI main page for this tablet" sec="P504412">P504412</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P504412" title="to CDLI main page for this tablet" sec="P504412">tablet P504412</a>|<span  class="hl" >face obverse</span>|<span  class="hldot hlup" >column 1</span>|<span  class="hldot hlup" >line 3</span>|<span  class="hl hlup" >TUN3~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/TUN3~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-tun3-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
18| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P504412" title="to CDLI main page for this tablet" sec="P504412">P504412</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P504412" title="to CDLI main page for this tablet" sec="P504412">tablet P504412</a>|<span  class="hl" >face obverse</span>|<span  class="hldot hlup" >column 1</span>|<span  class="hldot hlup" >line 5</span>|<span  class="hl hlup" >MASZ2</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/MASZ2.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-masz2.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
19| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P504412" title="to CDLI main page for this tablet" sec="P504412">P504412</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P504412" title="to CDLI main page for this tablet" sec="P504412">tablet P504412</a>|<span  class="hl" >face obverse</span>|<span  class="hldot hlup" >column 1</span>|<span  class="hldot hlup" >line 6</span>|<span  class="hl hlup" >SILA3~d</span> 
20| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P504412" title="to CDLI main page for this tablet" sec="P504412">P504412</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P504412" title="to CDLI main page for this tablet" sec="P504412">tablet P504412</a>|<span  class="hl" >face obverse</span>|<span  class="hldot hlup" >column 2</span>|<span  class="hldot hlup" >line 3</span>|<span  class="hl hlup" >KU6~a</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/KU6~a.jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-ku6-a.jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>

### Pit numbers

The feature `excavation` gives you the number of the pit where a tablet is found. 
The syntax of pit numbers is a bit involved, here are a few possible values:

```
W 20497
W 20335,3
W 19948,10
W 20493,26
W 17890,b
W 17729,o
W 15920,b5
W 17729,aq
W 19548,a + W 19548,b
W 17729,cn + W 17729,eq
W 14337,a + W 14337,b + W 14337,c + W 14337,d + W 14337,e
Ashm 1928-445b
```

Let's assume we are interested in `SZITA~a1` signs occurring in cases of depth 1.
The following query finds them all:

In [64]:
query = '''
tablet
  case depth=1
    sign grapheme=SZITA variant=a1
'''
results = A.search(query)

  0.17s 78 results


Now we want to organize them by excavation number:

In [65]:
signPerPit = {}

for (tablet, case, sign) in sorted(results):
    pit = F.excavation.v(tablet) or 'no pit information'
    signPerPit.setdefault(pit, []).append(sign)

for pit in sorted(signPerPit):
    print(f'{pit:<30} {len(signPerPit[pit]):>2}')

Ashm 1926,562                   1
Ashm 1926,567                   1
Ashm 1926,569                  13
Ashm 1926,695+737+741           6
Ashm 1926,716+732               1
Ashm 1926,739                   1
W 14731,z                       1
W 14777,c                       4
W 15776,i                       1
W 15785,a2                      1
W 15833,a01 + W 15833,aa04      1
W 15897,b5                      1
W 15897,c26                     1
W 20274,001                     1
W 20274,043                     1
W 20274,095                     2
W 20274,119                     1
W 20327,01                      1
W 20327,03                      1
W 20511,01                      1
W 20511,02                      6
W 21157                         1
W 21194                         1
W 21733,1                       3
W 22100,01                      4
W 22100,03                      5
W 22101,1                       1
W 23950                         1
W 23973,01                      1
W 24033,05    

We can restrict results to those on tablets found in certain pits by constraining the search template.
If we are interested in pit `20274` we can use a regular expression that matches all 4 detailed pit numbers
based on `20274`.
So, we do not say 

```
excavation=20274
```
but 

```
excavation~20274
```

In [67]:
query = '''
tablet excavation~20274
  case depth=1
    sign grapheme=SZITA variant=a1
'''
results = A.search(query)
A.table(results, condensed=False, lineart=False)

  0.19s 5 results


n | p | tablet | case | sign
--- | --- | --- | --- | ---
1| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003617" title="to CDLI main page for this tablet" sec="P003617">P003617</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003617" title="to CDLI main page for this tablet" sec="P003617">tablet P003617</a>|<span  class="hl" >case 2b</span>|<span  class="hl hlup" >SZITA~a1</span>
2| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003499" title="to CDLI main page for this tablet" sec="P003499">P003499</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003499" title="to CDLI main page for this tablet" sec="P003499">tablet P003499</a>|<span  class="hl" >case 2a</span>|<span  class="hl hlup" >SZITA~a1</span>
3| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003541" title="to CDLI main page for this tablet" sec="P003541">P003541</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003541" title="to CDLI main page for this tablet" sec="P003541">tablet P003541</a>|<span  class="hl" >case 1b</span>|<span  class="hl hlup" >SZITA~a1</span>
4| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003593" title="to CDLI main page for this tablet" sec="P003593">P003593</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003593" title="to CDLI main page for this tablet" sec="P003593">tablet P003593</a>|<span  class="hl" >case 2a</span>|<span  class="hl hlup" >SZITA~a1</span>
5| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003593" title="to CDLI main page for this tablet" sec="P003593">P003593</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003593" title="to CDLI main page for this tablet" sec="P003593">tablet P003593</a>|<span  class="hl" >case 3b</span>|<span  class="hl hlup" >SZITA~a1</span>

Or if we want to restrict ourselves to pit numbers with a `W`, we can say:

In [68]:
query = '''
tablet excavation~W
  case depth=1
    sign grapheme=SZITA variant=a1
'''
results = A.search(query)

  0.19s 42 results


## Quantifiers in templates

So far we have seen only very positive templates.
They express what you want to see in the result.

It is also possible to state conditions about what you do not want to see in the results.

### Tablets without case divisions

Let's find all tablets in which all lines are undivided, i.e. lines without cases.

In [69]:
query = '''
tablet
/without/
  case
/-/
'''

The expression

```
/without/
template
/-/
```

is a [quantifier](https://dans-labs.github.io/text-fabric/Use/Search/#quantifiers).

It poses a condition on the preceding line in the template, in this case the `tablet`.
And the condition is that the template

```
tablet
  case
```

does not have results.

In [70]:
results = A.search(query)

  0.04s 5384 results


In [71]:
A.show(results, end=2)



**tablet** *1*





**tablet** *2*



Now let's find cases without numerals.

In [72]:
query = '''
case
/without/
  sign type=numeral
/-/
'''
results = A.search(query)

  0.20s 2833 results


We show a few.

In [73]:
A.show(results, end=2)



**tablet** *1*





**tablet** *2*



Now we can use this to get something more sophisticated: the tablets that do not have numerals in their cases. So only undivided lines may contain numerals.

Let's find tablets that do have cases, but just no cases with numerals.

In [74]:
query = '''
tablet
/where/
  case
/have/
  /without/
    sign type=numeral
  /-/
/-/
/with/
  case
/-/
'''

In [75]:
results = A.search(query) 

  0.02s 53 results


In [76]:
A.show(results, end=2)



**tablet** *1*





**tablet** *2*



Can we find such tablet which do have numerals on their undivided lines.

We show here a way to use the results of one query in another one: 
*custom sets*.

We put the set of tablets with cases but without numerals in cases in a set called `cntablet`.

We run the query again, but now in shallow mode, so that the result is a set.

By the way: read more about custom sets and shallow mode in the description of
[`A.search()`](https://dans-labs.github.io/text-fabric/Api/General/#search-api).

In [77]:
results = A.search(query, shallow=True)
customSets = dict(cntablet=results)

  0.02s 53 results


Now we can perform a very simple query for numerals on this set: we want tablets with numerals.
By restricting ourselves to this set, we now that these numerals must occur on undivided lines.

In [78]:
query = '''
cntablet
  sign type=numeral
'''
results = A.search(query, sets=customSets)

  0.20s 160 results


In [79]:
A.show(results, end=2)



**tablet** *1*





**tablet** *2*



We could have found these results by one query as well.
Judge for yourself which method causes the least friction.

In [80]:
query = '''
tablet
/without/
  case
    sign type=numeral
/-/
/with/
  case
/-/
  sign type=numeral
'''
results = A.search(query)
A.show(results, end=2)

  0.20s 160 results




**tablet** *1*





**tablet** *2*



## Search and hand-coding

Now we want to find all the ShinPP numerals.

In [81]:
shinPP = dict(
    N41=0.2,
    N04=1,
    N19=6,
    N46=60,
    N36=180,
    N49=1800,
)

shinPPPat = '|'.join(shinPP)

We make use of the fact that we can construct our template.

In [82]:
query = f'''
tablet
  sign grapheme={shinPPPat}
'''
results = A.search(query)
A.table(results, end=20)

  0.18s 1018 results


n | p | tablet | sign | sign | sign
--- | --- | --- | --- | --- | ---
1| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448701" title="to CDLI main page for this tablet" sec="P448701">P448701</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P448701" title="to CDLI main page for this tablet" sec="P448701">tablet P448701</a>|<span  class="hl" >2(N19)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N19).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n19).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >4(N41)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/4(N41).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-4(n41).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >1(N46)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N46).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n46).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
2| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006005" title="to CDLI main page for this tablet" sec="P006005">P006005</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P006005" title="to CDLI main page for this tablet" sec="P006005">tablet P006005</a>|<span  class="hl" >1(N04)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N04).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n04).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
3| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002329" title="to CDLI main page for this tablet" sec="P002329">P002329</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002329" title="to CDLI main page for this tablet" sec="P002329">tablet P002329</a>|<span  class="hl" >2(N19)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N19).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n19).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
4| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002342" title="to CDLI main page for this tablet" sec="P002342">P002342</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002342" title="to CDLI main page for this tablet" sec="P002342">tablet P002342</a>|<span  class="hl" >1(N36)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N36).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n36).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >2(N19)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N19).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n19).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
5| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002344" title="to CDLI main page for this tablet" sec="P002344">P002344</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002344" title="to CDLI main page for this tablet" sec="P002344">tablet P002344</a>|<span  class="hl" >1(N04)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N04).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n04).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
6| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002398" title="to CDLI main page for this tablet" sec="P002398">P002398</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002398" title="to CDLI main page for this tablet" sec="P002398">tablet P002398</a>|<span  class="hl" >1(N04)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N04).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n04).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
7| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002622" title="to CDLI main page for this tablet" sec="P002622">P002622</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002622" title="to CDLI main page for this tablet" sec="P002622">tablet P002622</a>|<span  class="hl" >5(N19)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/5(N19).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-5(n19).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >1(N46)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N46).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n46).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >4(N19)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/4(N19).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-4(n19).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
8| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002626" title="to CDLI main page for this tablet" sec="P002626">P002626</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P002626" title="to CDLI main page for this tablet" sec="P002626">tablet P002626</a>|<span  class="hl" >1(N41)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N41).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n41).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
9| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003330" title="to CDLI main page for this tablet" sec="P003330">P003330</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003330" title="to CDLI main page for this tablet" sec="P003330">tablet P003330</a>|<span  class="hl" >3(N46)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/3(N46).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-3(n46).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >2(N49)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N49).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n49).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >5(N19)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/5(N19).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-5(n19).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >2(N04)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N04).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n04).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >1(N41)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N41).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n41).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
10| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003357" title="to CDLI main page for this tablet" sec="P003357">P003357</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003357" title="to CDLI main page for this tablet" sec="P003357">tablet P003357</a>|<span  class="hl" >1(N04)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N04).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n04).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
11| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003542" title="to CDLI main page for this tablet" sec="P003542">P003542</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003542" title="to CDLI main page for this tablet" sec="P003542">tablet P003542</a>|<span  class="hl" >1(N04)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N04).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n04).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
12| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003695" title="to CDLI main page for this tablet" sec="P003695">P003695</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003695" title="to CDLI main page for this tablet" sec="P003695">tablet P003695</a>|<span  class="hl" >1(N36)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N36).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n36).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >7(N19)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/7(N19).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-7(n19).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >1(N46)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N46).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n46).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >6(N19)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/6(N19).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-6(n19).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
13| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003810" title="to CDLI main page for this tablet" sec="P003810">P003810</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003810" title="to CDLI main page for this tablet" sec="P003810">tablet P003810</a>|<span  class="hl" >1(N04)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N04).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n04).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
14| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P004553" title="to CDLI main page for this tablet" sec="P004553">P004553</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P004553" title="to CDLI main page for this tablet" sec="P004553">tablet P004553</a>|<span  class="hl" >1(N04)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N04).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n04).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
15| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P004748" title="to CDLI main page for this tablet" sec="P004748">P004748</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P004748" title="to CDLI main page for this tablet" sec="P004748">tablet P004748</a>|<span  class="hl" >1(N04)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N04).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n04).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
16| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P004831" title="to CDLI main page for this tablet" sec="P004831">P004831</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P004831" title="to CDLI main page for this tablet" sec="P004831">tablet P004831</a>|<span  class="hl" >2(N46)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N46).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n46).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
17| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P004856" title="to CDLI main page for this tablet" sec="P004856">P004856</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P004856" title="to CDLI main page for this tablet" sec="P004856">tablet P004856</a>|<span  class="hl" >1(N04)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N04).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n04).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
18| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P004880" title="to CDLI main page for this tablet" sec="P004880">P004880</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P004880" title="to CDLI main page for this tablet" sec="P004880">tablet P004880</a>|<span  class="hl" >1(N04)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N04).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n04).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
19| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P005402" title="to CDLI main page for this tablet" sec="P005402">P005402</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P005402" title="to CDLI main page for this tablet" sec="P005402">tablet P005402</a>|<span  class="hl" >1(N49)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N49).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n49).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >1(N49)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N49).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n49).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >1(N19)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N19).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n19).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>
20| <a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P212396" title="to CDLI main page for this tablet" sec="P212396">P212396</a>|<a href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P212396" title="to CDLI main page for this tablet" sec="P212396">tablet P212396</a>|<span  class="hl" >1(N41)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N41).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n41).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >1(N41)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N41).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n41).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >1(N04)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N04).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n04).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >2(N41)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N41).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n41).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >1(N41)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N41).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n41).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >2(N04)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/2(N04).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-2(n04).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >4(N41)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/4(N41).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-4(n41).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>|<span  class="hl" >1(N04)</span> <a target="_blank" href="https://cdli.ucla.edu/tools/SignLists/protocuneiform/archsigns/1(N04).jpg" title="to higher resolution lineart on CDLI"><img src="cdli-imagery/lineart-1(n04).jpg" style="display: inline;max-width: 2em; max-height: 4em;"  /></a>

Let's see a few tablets in more detail:

In [83]:
A.show(results, end=5)



**tablet** *1*





**tablet** *2*





**tablet** *3*





**tablet** *4*





**tablet** *5*



### A tablet calculator

Rather than displaying search results, you can also *process* them in your program.

Search results come as tuples of nodes that correspond directly to the elements
of your search template.

We query for shinPP numerals on the faces of tablets.
The result of the query is a list of tuples `(t, f, s)` consisting of
a tablet node, a face node and a node for a sign of a shinPP numeral.

#### Rationale
This task will require a higher level of programming skills and a deeper knowledge of how
Python works.
We include it in this tutorial to get the message across that Text-Fabric is not
a black box that shields you from your data. Everything you handle in Text-Fabric is 
open to further programming and processing of your own design and choosing.

#### Data collection

In [84]:
query = f'''
tablet
    face
        sign type=numeral grapheme={shinPPPat}
'''
results = A.search(query)

  0.17s 1018 results


We are going to put all these numerals in buckets: for each face on each tablet a separate bucket.

In [85]:
numerals = {}
pNums = {}
for (tablet, face, sign) in results:
    pNums[F.catalogId.v(tablet)] = tablet
    numerals.setdefault(tablet, {}).setdefault(face, []).append(sign)
print(f'{len(pNums)} tablets')
print('\n'.join(list(pNums)[0:10]))
print('...')

235 tablets
P448701
P006005
P002329
P002342
P002344
P002398
P002622
P002626
P003330
P003357
...


#### The calculator
We define a function that given a tablet, adds the shinPP numerals by its faces.
We also show the line art and a pretty transcription.

The function is a bit involved.

In [86]:
# we generate Markdown strings and send them to the notebook formatter

def dm(x): display(Markdown(x))

def calcTablet(pNum): # pNum identifies the tablet in question
    # show a horizontal line in Markdown
    dm('---\n')     
    tablet = pNums.get(pNum, None)  # look up the node for this p-number
    if tablet is None:
        dm(f'**no results for {pNum}**')
        return                      # if not found the tablet has no ShinPP numerals: quit
    
    A.lineart(tablet, withCaption="top", width="200")   # show lineart
    faces = numerals[tablet]                    # get the buckets for the faces
    mySigns = []
    for (face, signs) in faces.items():         # work per face 
        mySigns.extend(signs)
        dm(f'##### {F.type.v(face)}')           # show the name of the face
        distinctSigns = {}                      # collect the distinct numerals
        for s in signs:
            distinctSigns.setdefault(A.atfFromSign(s), []).append(s)
        A.lineart(distinctSigns)      # display the list of signs
        total = 0                               # start adding up
        for (signAtf, signs) in distinctSigns.items():
            value = 0
            for s in signs:
                value += F.repeat.v(s) * shinPP[F.grapheme.v(s)]
            total += value
            amount = len(signs)                 # we report our calculation
            shinPPval = shinPP[F.grapheme.v(signs[0])]
            repeat = F.repeat.v(signs[0])
            print(f'{amount} x {signAtf} = {amount} x {repeat} x {shinPPval} = {value}')
        dm(f'**total** = **{total}**')
    A.prettyTuple([tablet] + mySigns, 1)                           # show pretty transcription

#### Calculate once

In [87]:
calcTablet('P006377')

---


##### obverse

1 x 1(N46) = 1 x 1 x 60 = 60
1 x 5(N19) = 1 x 5 x 6 = 30
4 x 3(N04) = 4 x 3 x 1 = 12
2 x 1(N41) = 2 x 1 x 0.2 = 0.4
8 x 1(N19) = 8 x 1 x 6 = 48
2 x 3(N19) = 2 x 3 x 6 = 36
5 x 1(N04) = 5 x 1 x 1 = 5
3 x 2(N04) = 3 x 2 x 1 = 6
3 x 2(N19) = 3 x 2 x 6 = 36
1 x 2(N41) = 1 x 2 x 0.2 = 0.4
2 x 4(N04) = 2 x 4 x 1 = 8
1 x 3(N41) = 1 x 3 x 0.2 = 0.6000000000000001
1 x 4(N19) = 1 x 4 x 6 = 24


**total** = **266.4**

##### reverse

1 x 1(N36) = 1 x 1 x 180 = 180
1 x 1(N46) = 1 x 1 x 60 = 60
1 x 8(N19) = 1 x 8 x 6 = 48
1 x 5(N04) = 1 x 5 x 1 = 5
1 x 3(N41) = 1 x 3 x 0.2 = 0.6000000000000001


**total** = **293.6**



**Result** *1*



#### Calculate ad lib
Now the first 5 tablets.

In [88]:
for tablet in sorted(pNums)[0:5]:
    calcTablet(tablet)

---


##### obverse

1 x 1(N04) = 1 x 1 x 1 = 1


**total** = **1**



**Result** *1*



...

---


##### obverse

1 x 1(N36) = 1 x 1 x 180 = 180


**total** = **180**



**Result** *1*



## More ...

The capabilities of search are endless.
Often it is the quickest way to focus on a phenomenon, quicker than hand coding all the logic
to retrieve your patterns.

That said, it is not a matter of either-or. You can use coding to craft your templates,
and you can use coding to process your results.

It's an explosive mix. A later chapter in this tutorial shows
even more [cases](cases.ipynb).

Have another look at
[the manual](https://dans-labs.github.io/text-fabric/Use/Search/).

# Next

[signs](signs.ipynb)

*Back to the basics ...*

All chapters:
[start](start.ipynb)
[imagery](imagery.ipynb)
[steps](steps.ipynb)
[search](search.ipynb)
[signs](signs.ipynb)
[quads](quads.ipynb)
[jumps](jumps.ipynb)
[cases](cases.ipynb)